In [10]:
# %% Import libs
import numpy as np
import types
import sys,time
import h5py
import itertools
import imageio
import copy
import re

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.pyplot as plt

from scipy.linalg import expm
from math import factorial
from copy import deepcopy
from mpl_toolkits.mplot3d import Axes3D
from scipy.special import sph_harm
from matplotlib.colors import Normalize


# %% Defining functions
# σ matrices

%matplotlib auto

I = np.array([[1,0],[0,1]])
sigma_x = np.array([[0,1],[1,0]])
sigma_y = np.array([[0,-1j], [1j,0]])
sigma_z = np.array([[1,0],[0,-1]])
n = np.array([[0,0],[0,1]])

def kronecker_delta(x,y):
    if abs(x-y)<1e-5: return 1
    else: return 0



# Defining Physical constants
hbar = 6.62607015*(10**(-34))/2/np.pi   # Yet I'd like to set hbar = 1 in the calculation follows
# Defining Atomic Constants
Gamma = 2*np.pi*6*10**6     # 6MHz linewidth of the ^{87}Rb upper state 5P3/2
kappa = 2*np.pi*0.1*10**6   # 0.1MHz linewidth of the cavity
Omega_a = 2*np.pi*6.834*10**9 # 6.83Ghz energy gap for up and down state

def husimi_q(state, theta, phi):
    q_func = np.zeros((len(theta), len(phi)), dtype=np.complex128)
    for j, t in enumerate(theta):
        for k, p in enumerate(phi):
            alpha = np.cos(t / 2) * np.exp(1j * p / 2)
            beta = np.sin(t / 2) * np.exp(-1j * p / 2)
            coherent_state = np.array([alpha ** m * beta ** (atom_number - m) * np.sqrt(factorial(atom_number) / (factorial(m) * factorial(atom_number - m))) for m in range(atom_number + 1)]).reshape(-1, 1)
            q_func[k, j] = np.abs(np.conj(coherent_state).T @ state) ** 2
    return np.abs(q_func)

# typ: Sx or Sz
# order: corresponding to the light frequency
# delta: AC stark shift
class photons():
    def __init__(self,typ,order,delta,t):
        self.order = order
        delta[np.abs(delta)<1] = 0
        self.delta = delta*1.0
        self.t = t
        self.typ = typ

class sequence():
    def __init__(self,photons):
        self.photons = photons



class cavity_atom_light_system():   # Note that here most of the parameters are alterable, except for the atomic parameters.
    def __init__(self, atom_number, cooperativity, Delta, Omega):
        self.atom_number = atom_number              
        self.cooperativity = cooperativity              
        self.Delta = Delta                              # Delta should be much smaller than 6.8GHz/2, otherwise self.t() no longer holds
        self.omega_s = self.cooperativity*kappa*Gamma / (4*self.Delta)     # omega_s = g^2/Δ = ηκΓ/(4Δ)
        self.Omega = Omega                              # Note that we set Omega as a fixed parameter here.
        self.state_index = np.arange(self.atom_number+1)[::-1]    # The first one being the Dicke state with spin_z==+N/2
        # Calculating S_xyz, be ware of the order of Dicke states
        self.S_z = np.diag(np.array([self.atom_number/2 - iter for iter in np.arange(atom_number+1)]))
        S_plus  = np.array( [ [np.sqrt(atom_number/2 * (atom_number/2 +1) - jj * ( jj + 1 ) ) * kronecker_delta(ii, jj+1) for jj in np.arange(atom_number/2,-atom_number/2-1,-1)] for ii in np.arange(atom_number/2,-atom_number/2-1,-1)] )
        S_minus = np.array( [ [np.sqrt(atom_number/2 * (atom_number/2 +1) - jj * ( jj - 1 ) ) * kronecker_delta(ii, jj-1) for jj in np.arange(atom_number/2,-atom_number/2-1,-1)] for ii in np.arange(atom_number/2,-atom_number/2-1,-1)] )
        self.S_x = (S_plus + S_minus) / 2
        self.S_y = (S_plus - S_minus) / (2j)

        self.psi0 = self.initialize_psi0()
        self.psi = deepcopy(self.psi0)  # Remember to reset psi after each evolution turn

    def initialize_psi0(self):      # Initialize a CSS orienting at the north pole, that is, [1,0,0,0,...,0] in the Dicke representation
        psi0 = np.zeros(self.atom_number+1)
        psi0[0] = 1
        return psi0.reshape(-1,1)     # State vectors are column vectors, of course
    
    def H_AC(self, delta, order): # "delta" is the depth of the AC stark shift, and "order" is the Dicke state order with which the AC light resonates
        H_prime_exp = delta*(1 - 1j*Gamma / (2*self.Delta) ) * np.diag([n*np.abs(self.t(order*self.omega_s, n))**2 for n in range(self.atom_number+1)[::-1]]) / (order * np.abs(self.t(order * self.omega_s, order))**2 )   # Check that the first one being +N/2
        return H_prime_exp  # order = 1 recovers the Eq.6 in the main text of Creation of Greenberger-Horne-Zeilinger states with thousands of atoms by entanglement amplification

    def show_H_AC_photons(self,sequence):
        photons = sequence.photons
        sequence_length = len(sequence.photons)
        H_ACs = np.array([[0 for i in range(self.atom_number+1)]],dtype = float)
        for i in range(sequence_length):
            H_AC = self.H_AC(photons[i].delta[0], photons[i].order[0])*0
            for j in range(len(photons[i].delta)):
                H_AC += self.H_AC(photons[i].delta[j], photons[i].order[j])
            H_ACs = np.concatenate((H_ACs,np.array([np.diagonal(H_AC)])))
        H_ACs = np.delete(H_ACs,0,0)
        return H_ACs
    
        
    def evolve_photons(self,sequence):
        photons = sequence.photons
        sequence_length = len(sequence.photons)
        for i in range(sequence_length):
            H_AC = self.H_AC(photons[i].delta[0], photons[i].order[0])*0
            for j in range(len(photons[i].delta)):
                H_AC += self.H_AC(photons[i].delta[j], photons[i].order[j])
            if photons[i].typ == "x":
                self.psi = expm(-1j*(self.Omega*self.S_x + H_AC)*photons[i].t)@ self.psi
            elif photons[i].typ == "z":
                self.psi = expm(-1j*(self.Omega*self.S_z + H_AC)*photons[i].t)@ self.psi
            else:
                return "Error, wrong input for S type."
    
    def evolve_photons_t(self,sequence,t_view):
        self.system_reset()
        photons = sequence.photons
        sequence_length = len(photons)
        t0 = 0
        marker = 0
        for i in range(sequence_length):
            if t_view>t0:
                t0 += photons[i].t
                marker += 1
            else:
                break
        if marker != 0:
            for i in range(marker-1):
                H_AC = self.H_AC(photons[i].delta[0], photons[i].order[0])*0
                for j in range(len(photons[i].delta)):
                    H_AC += self.H_AC(photons[i].delta[j], photons[i].order[j])
                if photons[i].typ == "x":
                    self.psi = expm(-1j*(self.Omega*self.S_x + H_AC)*photons[i].t)@ self.psi
                elif photons[i].typ == "z":
                    self.psi = expm(-1j*(self.Omega*self.S_z + H_AC)*photons[i].t)@ self.psi
                else:
                    return "Error, wrong input for S type."
            H_AC = self.H_AC(photons[marker-1].delta[0], photons[i].order[0])*0
            t_pre = 0
            for j in range(marker-1):
                t_pre += photons[j].t
            for j in range(len(photons[marker-1].delta)):
                H_AC += self.H_AC(photons[marker-1].delta[j], photons[marker-1].order[j])
            if photons[marker-1].typ == "x":
                self.psi = expm(-1j*(self.Omega*self.S_x + H_AC)*(t_view-t_pre))@ self.psi
            elif photons[marker-1].typ == "z":
                self.psi = expm(-1j*(self.Omega*self.S_z + H_AC)*(t_view-t_pre))@ self.psi
        theta = np.linspace(0, np.pi, 100)
        phi = np.linspace(0, np.pi*4/2, 100)

        return husimi_q(self.psi,theta,phi)
    
    
    
    def evolve_typeC_params(self,params):
        t_x1, t_z, t_x2, order, delta, t  = params
        self.psi = expm(-1j*self.Omega*self.S_x*t_x2)@expm(-1j*self.Omega*self.S_z*t_z)@expm(-1j*self.Omega*self.S_x*t_x1)@expm(-1j * (self.Omega*self.S_x + self.H_AC(delta, order)) * t) @ self.psi
        # self.show_occupation()
    
    def evolve_typeC_params_t(self, params, t_view):
        t_x1, t_z, t_x2, order, delta, t  = params
        if t_view<0:
            psi = self.psi
        elif t_view<t:
            psi = expm(-1j * (self.Omega*self.S_x + self.H_AC(delta, order)) * t_view) @ self.psi
        elif t_view<t+t_x1:
            psi = expm(-1j*self.Omega*self.S_x*(t_view-t))@expm(-1j * (self.Omega*self.S_x + self.H_AC(delta, order)) * t) @ self.psi
        elif t_view<t+t_x1+t_z:
            psi = expm(-1j*self.Omega*self.S_z*(t_view-t-t_x1))@expm(-1j*self.Omega*self.S_x*t_x1)@expm(-1j * (self.Omega*self.S_x + self.H_AC(delta, order)) * t) @ self.psi
        elif t_view<t+t_x1+t_z+t_x2:
            psi = expm(-1j*self.Omega*self.S_x*(t_view-t-t_x1-t_z))@expm(-1j*self.Omega*self.S_z*t_z)@expm(-1j*self.Omega*self.S_x*t_x1)@expm(-1j * (self.Omega*self.S_x + self.H_AC(delta, order)) * t) @ self.psi
        else:
            psi = expm(-1j*self.Omega*self.S_x*t_x2)@expm(-1j*self.Omega*self.S_z*t_z)@expm(-1j*self.Omega*self.S_x*t_x1)@expm(-1j * (self.Omega*self.S_x + self.H_AC(delta, order)) * t) @ self.psi
        theta = np.linspace(0, np.pi, 100)
        phi = np.linspace(0, np.pi*4/2, 100)
        return husimi_q(psi,theta,phi)

            
    def evaluation(self, mode = "SSS"):
        if mode == "GHZ":
            density_mat = self.psi@np.transpose(np.conj(self.psi))
            fidelity = 1/2 * np.abs(density_mat[0,0] + density_mat[self.atom_number, self.atom_number] + np.abs(density_mat[0, self.atom_number] + density_mat[self.atom_number, 0]) )
            return fidelity
        elif mode == "SSS": 
            # This is ξ_R^2, which is sensitive to the unitarity for the most.
            rho = self.psi@np.transpose(np.conj(self.psi))
            loss_atom_num = (1-np.real(np.trace(rho)))*self.atom_number
            Jzmean = np.real(np.trace(self.S_z@rho))
            Jxmean = np.real(np.trace(self.S_x@rho))
            Jymean = np.real(np.trace(self.S_y@rho))
            Jabs = np.real(np.sqrt(Jzmean**2 + Jxmean**2 + Jymean**2))
            Polar_Angle = np.arccos(Jzmean/Jabs)
            if Polar_Angle == 0:
                Azimuthal_Angle = 0
            else:
                temp = Jxmean/(Jabs*np.sin(Polar_Angle))
                if temp>1:temp-=1e-6
                if temp<-1:temp+=1e-6
                Azimuthal_Angle = np.arccos(temp)
                if Jymean<=0: Azimuthal_Angle = 2*np.pi - Azimuthal_Angle
            n_1 = np.array([-np.sin(Azimuthal_Angle), np.cos(Azimuthal_Angle), 0])
            n_2 = np.array([np.cos(Polar_Angle)*np.cos(Azimuthal_Angle), np.cos(Polar_Angle)*np.sin(Azimuthal_Angle), -np.sin(Polar_Angle)])
            J_n1 = self.S_x*n_1[0] + self.S_y*n_1[1] + self.S_z*n_1[2]
            J_n2 = self.S_x*n_2[0] + self.S_y*n_2[1] + self.S_z*n_2[2]
            C = np.real(np.trace(rho@(J_n1@J_n1+J_n2@J_n2)))
            A = np.real(np.trace(rho@(J_n1@J_n1 - J_n2@J_n2)))
            B = np.real(np.trace(rho@J_n1@J_n2+rho@J_n2@J_n1))
            min_variance = 1/2*(C-np.sqrt(A**2+B**2))
            variance = min_variance +1/4*np.sqrt(loss_atom_num)
            Squeezing_Parameter = self.atom_number/(Jabs**2) * variance
            
            return Squeezing_Parameter, Jabs, Polar_Angle, Azimuthal_Angle
        elif mode == "SSSN": 
            # This is ξ_R^2, which is sensitive to the unitarity for the most.
            rho = self.psi@np.transpose(np.conj(self.psi))
            loss_atom_num = (1-np.real(np.trace(rho)))*self.atom_number
            Jzmean = np.real(np.trace(self.S_z@rho))
            Jxmean = np.real(np.trace(self.S_x@rho))
            Jymean = np.real(np.trace(self.S_y@rho))
            Jabs = np.real(np.sqrt(Jzmean**2 + Jxmean**2 + Jymean**2))
            Polar_Angle = np.arccos(Jzmean/Jabs)
            if Polar_Angle == 0:
                Azimuthal_Angle = 0
            else:
                temp = Jxmean/(Jabs*np.sin(Polar_Angle))
                if temp>1:temp-=1e-6
                if temp<-1:temp+=1e-6
                Azimuthal_Angle = np.arccos(temp)
                if Jymean<=0: Azimuthal_Angle = 2*np.pi - Azimuthal_Angle
            n_1 = np.array([-np.sin(Azimuthal_Angle), np.cos(Azimuthal_Angle), 0])
            n_2 = np.array([np.cos(Polar_Angle)*np.cos(Azimuthal_Angle), np.cos(Polar_Angle)*np.sin(Azimuthal_Angle), -np.sin(Polar_Angle)])
            J_n1 = self.S_x*n_1[0] + self.S_y*n_1[1] + self.S_z*n_1[2]
            J_n2 = self.S_x*n_2[0] + self.S_y*n_2[1] + self.S_z*n_2[2]
            C = np.real(np.trace(rho@(J_n1@J_n1+J_n2@J_n2)))
            A = np.real(np.trace(rho@(J_n1@J_n1 - J_n2@J_n2)))
            B = np.real(np.trace(rho@J_n1@J_n2+rho@J_n2@J_n1))
            min_variance = 1/2*(C-np.sqrt(A**2+B**2))
            variance = min_variance
            Squeezing_Parameter = np.real(np.trace(rho))*self.atom_number/(Jabs**2) * variance
            
            return Squeezing_Parameter, Jabs, Polar_Angle, Azimuthal_Angle
        else:
            p = re.compile('(N\d*)*')
            m = p.match(mode)
            if not m==None:
                rho = self.psi@np.transpose(np.conj(self.psi))
                nums = np.array([],dtype = int)
                pnum = re.compile("\d\d*")
                endmark = 0
                counter = 0
                while endmark<(len(mode)-1):
                    mnum = pnum.search(mode,endmark)
                    num = mode[mnum.span()[0]:mnum.span()[1]]
                    nums = np.append(nums,int(num))
                    endmark = mnum.span()[1]
                    counter += 1
                fidelity = 0.0
                for i in range(counter):
                    for j in range(counter):
                        fidelity += np.real(rho[nums[i],nums[j]])/counter
                return 1-fidelity,0


    # Cavity amplitude transmission function t
    def t(self, ksi, n_up):   # ksi is the detuning between light and cavity mode ω-ω_c
        return 1 / ( 1 + (n_up*self.cooperativity) / (1+4*(self.Delta+ksi)**2/Gamma**2) + ((self.atom_number-n_up)*self.cooperativity) / (1+4*(-Omega_a+self.Delta+ksi)**2/Gamma**2) - 2*1j* ( ksi/kappa - n_up*self.cooperativity*(self.Delta+ksi)/Gamma / (1+4*(self.Delta+ksi)**2/Gamma**2) - (self.atom_number-n_up)*self.cooperativity*(-Omega_a+self.Delta+ksi)/Gamma / (1+4*(-Omega_a+self.Delta+ksi)**2/Gamma**2) ) )

    def show_occupation(self):
        plt.figure()
        plt.scatter(self.state_index-self.atom_number/2, np.abs(np.transpose(self.psi))**2)     # Occupation is the probability distribution, for sure
        plt.show()

    def system_reset(self):
        self.psi = deepcopy(self.psi0)

    def husimi_q(self, theta, phi):
        q_func = np.zeros((len(theta), len(phi)), dtype=np.complex128)
        for j, t in enumerate(theta):
            for k, p in enumerate(phi):
                alpha = np.cos(t / 2) * np.exp(1j * p / 2)
                beta = np.sin(t / 2) * np.exp(-1j * p / 2)
                coherent_state = np.array([alpha ** m * beta ** (self.atom_number - m) * np.sqrt(factorial(self.atom_number) / (factorial(m) * factorial(self.atom_number - m))) for m in range(self.atom_number + 1)]).reshape(-1, 1)
                q_func[k, j] = np.abs(np.conj(coherent_state).T @ self.psi) ** 2
        return np.abs(q_func)

    def visualize(self):
        theta = np.linspace(0, np.pi, 100)
        phi = np.linspace(0, np.pi*4/2, 100)
        theta_grid, phi_grid = np.meshgrid(theta, phi)
        q_func = self.husimi_q(theta, phi)
        x = np.sin(theta_grid) * np.cos(phi_grid)
        y = np.sin(theta_grid) * np.sin(phi_grid)
        z = -np.cos(theta_grid)
        # Normalize the q_func values for the colormap
        norm = Normalize(vmin=np.min(q_func), vmax=np.max(q_func))
        

        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.plot_surface(x, y, z, rstride=1, cstride=1, facecolors=plt.cm.jet(norm(q_func)), alpha=0.3, linewidth=0, antialiased=False)
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_title('Husimi-Q Representation')
#         ax.set_aspect('equal')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_zticks([])
        plt.axis('off')
        plt.show()
        

        
def find_H_AC(atom_number,ini_diag = np.array([None,None]),ini_t = np.array([None,None]),times = 1000):
    T0 = 0.003
    T = T0
    system = cavity_atom_light_system(atom_number = atom_number, cooperativity = 100, Delta = -2*np.pi*4*10**9, Omega = 0.2*10**6*2*np.pi)
    if (ini_diag == None).all():
        ini_diag = -np.ones(atom_number+1,dtype = complex)*10**6
    if (ini_t == None).all():
        ini_t = np.array([1.0*10**(-6),0.6*10**(-6)])
    system.psi = expm(-1j*(system.Omega*system.S_x)*ini_t[0])@ system.psi
    system.psi = expm(-1j*(system.Omega*system.S_x + np.diag(ini_diag))*ini_t[1])@ system.psi
    ini_Xr = system.evaluation(mode = "SSSN")[0]
    grad_len_ratio = 1.0/times
    step_len_ratio = 10.0/times
    for time in range(times):
        dXr = 0
        for order in range(atom_number+1):
            new_dXr = 0
            direction = np.random.randint(2)
            system.system_reset()
            grad_diag = ini_diag*1.0
            grad_diag[int(np.mod(order*((-1)**(direction)),atom_number+1))] = grad_diag[int(np.mod(order*((-1)**(direction)),atom_number+1))]*(1+grad_len_ratio)
            system.psi = expm(-1j*(system.Omega*system.S_x)*ini_t[0])@ system.psi
            system.psi = expm(-1j*(system.Omega*system.S_x + np.diag(grad_diag))*ini_t[1])@ system.psi
            grad_Xr = system.evaluation(mode = "SSSN")[0]
            system.system_reset()
            sign = -np.sign(grad_Xr-ini_Xr)
            new_diag = ini_diag*1.0
            new_diag[int(np.mod(order*((-1)**(direction)),atom_number+1))] = new_diag[int(np.mod(order*((-1)**(direction)),atom_number+1))]*(1+step_len_ratio*sign*np.random.rand())
            system.psi = expm(-1j*(system.Omega*system.S_x)*ini_t[0])@ system.psi
            system.psi = expm(-1j*(system.Omega*system.S_x + np.diag(new_diag))*ini_t[1])@ system.psi
            new_Xr = system.evaluation(mode = "SSSN")[0]
            system.system_reset()
            if new_Xr<ini_Xr:
                T = T/(1+100/times/atom_number)
                new_dXr = ini_Xr-new_Xr
                ini_Xr = deepcopy(new_Xr)
                ini_diag = new_diag*1.0
            else:
                Tjudge = np.exp(-(new_Xr-ini_Xr)/T)
                random = np.random.rand()
                if random<Tjudge:
                    T = T/(1+100/times/atom_number)
                    ini_Xr = deepcopy(new_Xr)
                    ini_diag = new_diag*1.0
            if new_dXr>dXr:
                dXr = new_dXr*1.0
            t_random = np.random.randint(2)
            grad_t = ini_t*1.0
            grad_t[t_random]*=(1+grad_len_ratio)
            system.psi = expm(-1j*(system.Omega*system.S_x)*grad_t[0])@ system.psi
            system.psi = expm(-1j*(system.Omega*system.S_x + np.diag(ini_diag))*grad_t[1])@ system.psi
            grad_Xr = system.evaluation(mode = "SSSN")[0]
            system.system_reset()
            sign = -np.sign(grad_Xr-ini_Xr)
            new_t = ini_t*1.0
            new_t[t_random]*=(1+step_len_ratio*sign*np.random.rand())
            system.psi = expm(-1j*(system.Omega*system.S_x)*new_t[0])@ system.psi
            system.psi = expm(-1j*(system.Omega*system.S_x + np.diag(ini_diag))*new_t[1])@ system.psi
            new_Xr = system.evaluation(mode = "SSSN")[0]
            system.system_reset()
            if new_Xr<ini_Xr:
                T = T/(1+100/times/atom_number)
                new_dXr = ini_Xr-new_Xr
                ini_Xr = deepcopy(new_Xr)
                ini_t = new_t*1.0
            else:
                Tjudge = np.exp(-(new_Xr-ini_Xr)/T)
                random = np.random.rand()
                if random<Tjudge:
                    T = T/(1+100/times/atom_number)
                    ini_Xr = deepcopy(new_Xr)
                    ini_t = new_t*1.0
            if new_dXr>dXr:
                dXr = new_dXr*1.0
            print("\r", end="")
            print(f"Scan finished:{time+1}/{times}, Xr = {ini_Xr}, order = {order}, dXr = {dXr}", end="")
            sys.stdout.flush()
        if dXr < ini_Xr*10**(-5):
            T = T*(1+25/atom_number)
    return ini_diag,ini_t

# Genetic evolution
def genetic_evolution(generations,size,cooperativity, atom_number, ini_Delta, target, inisequence = None,seq_evol = 0,pho_evol = 1,Delta_evol = 1):
    system = cavity_atom_light_system(atom_number = atom_number, cooperativity = cooperativity, Delta = ini_Delta, Omega = 0.2*10**6*2*np.pi)
    if inisequence == None:
        old_generation = np.array([sequence([photons("x",np.array([atom_number/2]),np.array([-100*10**6]),1.0*10**(-6))]) for i in range(100)])
    
    else:
        old_generation = np.array([inisequence for i in range(size)])
    old_Delta = np.array([ini_Delta for i in range(size)])

    
    old_behavior = np.ones(size,dtype = float)
    old_phnum = np.array([])
    old_seqnum = np.array([])
    for i in range(size):
        system.Delta = old_Delta[i]
        system.psi = deepcopy(system.psi0)
        system.omega_s = system.cooperativity*kappa*Gamma / (4*system.Delta)
        system.evolve_photons(old_generation[i])
        Xr, Jabs, polartheta, polorphi = system.evaluation(mode = target)
        old_behavior[i] = Xr
        old_seqnum = np.append(old_seqnum,len(old_generation[i].photons))
        for pho in range(len(old_generation[i].photons)):
            old_phnum = np.append(old_phnum,len(old_generation[i].photons[pho].delta))
    for evolve in range(generations):

        new_generation = np.array([])
        if seq_evol == 1:
            Ps = 0.001
        else:
            Ps = 0
        if pho_evol ==1:
            Pp = 0.01
        else:
            Pp = 0
        step_len_ratio = 10**(-2)
        if Delta_evol == 1:
            new_Delta = old_Delta*((1-2*np.random.rand(size))*step_len_ratio+1)
        else:
            new_Delta = old_Delta*1.0
        new_behavior = np.ones(size,dtype = float)
        for i in range(size):
            news_judge = np.random.rand()
            newp_judge = np.random.rand()
            new_photons = np.array([])
            for seq in range(len(old_generation[i].photons)):
                l = len(old_generation[i].photons[seq].delta)
                delta = old_generation[i].photons[seq].delta*((1-2*np.random.rand(l))*step_len_ratio+1)
                order = old_generation[i].photons[seq].order*((1-2*np.random.rand(l))*step_len_ratio+1)
                if newp_judge<Pp:
                    delta = np.append(delta,delta[-1])*np.random.rand()*step_len_ratio
                    order = np.append(order,order[-1]*((5*np.random.rand())*step_len_ratio+1))
                order_sort = np.argsort(order)
                delete_ind = np.array([],dtype = int)
                for ind in range(len(order_sort)-1):
                    if order[order_sort[ind+1]]-order[order_sort[ind]]<atom_number/1000:
                        delete_ind = np.append(delete_ind,int(order_sort[ind+1]))
                        delta[order_sort[ind]] = delta[order_sort[ind]]+delta[order_sort[ind+1]]
                order = np.delete(order,delete_ind)
                delta = np.delete(delta,delete_ind)
                delta[np.abs(delta)<10**(6)] = 0
                t = old_generation[i].photons[seq].t*((1-2*np.random.rand())*step_len_ratio+1)
                typ = old_generation[i].photons[seq].typ
                new_photons = np.append(new_photons, photons(typ,order,delta,t))
            if news_judge>Ps:
                Ps = Ps*1.1
            else:
                seq = -1
                l = len(old_generation[i].photons[seq].delta)
                delta = old_generation[i].photons[seq].delta*((1-2*np.random.rand(l))*step_len_ratio+1)*step_len_ratio
                order = old_generation[i].photons[seq].order*((1-2*np.random.rand(l))*step_len_ratio+1)
                t = old_generation[i].photons[seq].t*((1-2*np.random.rand())*step_len_ratio+1)*step_len_ratio
                randtyp = np.random.randint(2)
                if randtyp == 1:
                    typ = "x"
                else:
                    typ = "z"
                    
                order_sort = np.argsort(order)
                delete_ind = np.array([],dtype = int)
                for ind in range(len(order_sort)-1):
                    if order[order_sort[ind+1]]-order[order_sort[ind]]<atom_number/1000:
                        delete_ind = np.append(delete_ind,int(order_sort[ind+1]))
                        delta[order_sort[ind]] = delta[order_sort[ind]]+delta[order_sort[ind+1]]
                order = np.delete(order,delete_ind)
                delta = np.delete(delta,delete_ind)
                new_photons = np.append(new_photons, photons(typ,order,delta,t))
                Ps = 0.001
            seq_delete_ind = np.array([],dtype = int)
            for seq in range(len(new_photons)-1):
                order_judge1 = (len(new_photons[seq].order) == len(new_photons[seq+1].order))
                delta_judge = False
                order_judge = False
                if order_judge1:
                    sort1 = np.argsort(new_photons[seq].order)
                    sort2 = np.argsort(new_photons[seq+1].order)
                    order_judge = (new_photons[seq].order[sort1] - new_photons[seq+1].order[sort2] < atom_number*1/500/len(new_photons[seq].order)).all()
                    delta_judge = (new_photons[seq].delta[sort1] - new_photons[seq+1].delta[sort2] < 1.0*10**6/len(new_photons[seq].order)).all()
                if delta_judge and order_judge:
                    seq_delete_ind = np.append(seq_delete_ind,seq)
                    new_photons[seq+1].t = new_photons[seq+1].t+new_photons[seq].t
            new_photons = np.delete(new_photons,seq_delete_ind)
            
            new_generation = np.append(new_generation,sequence(new_photons))
            system.Delta = new_Delta[i]
            system.omega_s = system.cooperativity*kappa*Gamma / (4*system.Delta)
            system.psi = deepcopy(system.psi0)
            system.evolve_photons(new_generation[i])
            Xr, Jabs, polartheta, polorphi = system.evaluation(mode = target)
            new_behavior[i] = Xr
        total_generation = np.concatenate((old_generation,new_generation))
        total_Delta = np.concatenate((old_Delta,new_Delta))
        total_behavior = np.concatenate((old_behavior,new_behavior))
        total_rank = np.argsort(total_behavior)
        for i in range(size):
            old_generation[i] = total_generation[total_rank[i]]
            old_Delta[i] = total_Delta[total_rank[i]]
            old_behavior[i] = total_behavior[total_rank[i]]
        old_phnum = np.array([])
        old_seqnum = np.array([])
        for i in range(size):
            old_seqnum = np.append(old_seqnum,len(old_generation[i].photons))
            for pho in range(len(old_generation[i].photons)):
                old_phnum = np.append(old_phnum,len(old_generation[i].photons[pho].delta))
        max_phnum = np.max(old_phnum)
        max_seqnum = np.max(old_seqnum)
        Xrbest = np.min(old_behavior)
        print("\r", end="")
        print(f"Evolution finished:{evolve}/{generations},max phnum = {max_phnum}, max seqnum = {max_seqnum} best Xr = {Xrbest}", end="")
        sys.stdout.flush()
    all_sort = np.argsort(old_behavior)
    return old_generation[all_sort[0]],old_Delta[all_sort[0]],old_behavior[all_sort[0]]

def multi_frequency_scan(etamax,atom_number, target,ini_Delta = None, inisequence = None, times = 1000, opt_Delta = False):
    T = 0.0003
    if ini_Delta == None:
        ini_Delta =  -4115000000.0
    ini_eta = etamax
    system = cavity_atom_light_system(atom_number = atom_number, cooperativity = ini_eta, Delta = ini_Delta, Omega = 0.2*10**6*2*np.pi)
    if inisequence == None:
        ini_sequence = sequence([photons("x",np.array([atom_number/2]),np.array([-100*10**6]),1.0*10**(-6))])
    else:
        ini_sequence = inisequence
        
    ini_photons = inisequence.photons
    nseq = len(ini_photons)
    
    system.evolve_photons(ini_sequence)
    res = system.evaluation(mode = target)[0]
    min_res = res*1.0
    system.system_reset()
    
    grad_len_ratio = 10.0/times
    step_len_ratio = 40.0/times
    
    for time in range(times):
        dres = 0
        for seq in range(nseq):
            ini_order = ini_photons[seq].order
            ini_delta = ini_photons[seq].delta
            ini_t = ini_photons[seq].t
            new_dres = 0
            for nm in range(len(ini_photons[seq].order)):
                new_order = ini_order*1.0
                grad_order = ini_order*1.0
                grad_delta = ini_delta*1.0
                grad_order[nm] = grad_order[nm]*(1+grad_len_ratio)
                
                grad_photons = np.array([],dtype = object)
                for ns in range(nseq):
                    if not ns == seq:
                        grad_photons = np.append(grad_photons,ini_photons[ns])
                    else:
                        grad_photons = np.append(grad_photons,photons("x",grad_order,grad_delta,ini_t))
                grad_sequence = sequence(grad_photons)

                system.evolve_photons(grad_sequence)
                grad_res = system.evaluation(mode = target)[0]
                system.system_reset()

                sign = -np.sign(grad_res-res)
                new_order[nm] = new_order[nm]*(1+step_len_ratio*sign*np.random.rand())
                new_delta = ini_delta*1.0
                
                new_photons = np.array([],dtype = object)
                for ns in range(nseq):
                    if not ns == seq:
                        new_photons = np.append(new_photons,ini_photons[ns])
                    else:
                        new_photons = np.append(new_photons,photons("x",new_order,new_delta,ini_t))
                new_sequence = sequence(new_photons)

                system.evolve_photons(new_sequence)
                new_res = system.evaluation(mode = target)[0]
                system.system_reset()

                if new_res<res:
                    T = T/(1+500/times/atom_number)
                    new_dres = res-new_res
                    res = deepcopy(new_res)
                    if res<min_res:
                        min_res = res*1.0
                    ini_order = new_order*1.0
                else:
                    Tjudge = np.exp(-(new_res-res)/T)
                    random = np.random.rand()
                    if random<Tjudge:
                        res = deepcopy(new_res)
                        ini_order = new_order*1.0
                if new_dres>dres:
                    dres = new_dres*1.0

                new_order = ini_order*1.0
                grad_order = ini_order*1.0
                grad_delta = ini_delta*1.0
                grad_delta[nm] = grad_delta[nm]*(1+grad_len_ratio)
                
                grad_photons = np.array([],dtype = object)
                for ns in range(nseq):
                    if not ns == seq:
                        grad_photons = np.append(grad_photons,ini_photons[ns])
                    else:
                        grad_photons = np.append(grad_photons,photons("x",grad_order,grad_delta,ini_t))
                grad_sequence = sequence(grad_photons)

                system.evolve_photons(grad_sequence)
                grad_res = system.evaluation(mode = target)[0]
                system.system_reset()

                sign = -np.sign(grad_res-res)
                new_delta[nm] = new_delta[nm]*(1+step_len_ratio*sign*np.random.rand())
                new_order = ini_order*1.0
                
                new_photons = np.array([],dtype = object)
                for ns in range(nseq):
                    if not ns == seq:
                        new_photons = np.append(new_photons,ini_photons[ns])
                    else:
                        new_photons = np.append(new_photons,photons("x",new_order,new_delta,ini_t))
                new_sequence = sequence(new_photons)


                system.evolve_photons(new_sequence)
                new_res = system.evaluation(mode = target)[0]
                system.system_reset()

                if new_res<res:
                    T = T/(1+500/times/atom_number)
                    new_dres = res-new_res
                    res = deepcopy(new_res)
                    if res<min_res:
                        min_res = res*1.0
                    ini_delta = new_delta*1.0
                else:
                    Tjudge = np.exp(-(new_res-res)/T)
                    random = np.random.rand()
                    if random<Tjudge:
                        res = deepcopy(new_res)
                        ini_delta = new_delta*1.0
                
                grad_t = ini_t*(1+grad_len_ratio)
                
                grad_photons = np.array([],dtype = object)
                for ns in range(nseq):
                    if not ns == seq:
                        grad_photons = np.append(grad_photons,ini_photons[ns])
                    else:
                        grad_photons = np.append(grad_photons,photons("x",ini_order,ini_delta,grad_t))
                grad_sequence = sequence(grad_photons)

                system.evolve_photons(grad_sequence)
                grad_res = system.evaluation(mode = target)[0]
                system.system_reset()

                sign = -np.sign(grad_res-res)
                new_t = ini_t*(1+step_len_ratio*sign*np.random.rand())
                
                new_photons = np.array([],dtype = object)
                for ns in range(nseq):
                    if not ns == seq:
                        new_photons = np.append(new_photons,ini_photons[ns])
                    else:
                        new_photons = np.append(new_photons,photons("x",ini_order,ini_delta,new_t))
                new_sequence = sequence(new_photons)


                system.evolve_photons(new_sequence)
                new_res = system.evaluation(mode = target)[0]
                system.system_reset()

                if new_res<res:
                    T = T/(1+500/times/atom_number)
                    new_dres = res-new_res
                    res = deepcopy(new_res)
                    if res<min_res:
                        min_res = res*1.0
                    ini_t = new_t*1.0
                else:
                    Tjudge = np.exp(-(new_res-res)/T)
                    random = np.random.rand()
                    if random<Tjudge:
                        res = deepcopy(new_res)
                        ini_t = new_t*1.0
                
                if opt_Delta:
                    Delta_ini_photons = np.array([],dtype = object)
                    for ns in range(nseq):
                        if not ns == seq:
                            Delta_ini_photons = np.append(Delta_ini_photons,ini_photons[ns])
                        else:
                            Delta_ini_photons = np.append(Delta_ini_photons,photons("x",ini_order,ini_delta,ini_t))
                    Delta_ini_sequence = sequence(grad_photons)

                    grad_Delta = ini_Delta*(1+grad_len_ratio)

                    system.Delta = grad_Delta
                    system.omega_s = system.cooperativity*kappa*Gamma / (4*system.Delta)

                    system.evolve_photons(Delta_ini_sequence)
                    grad_res = system.evaluation(mode = target)[0]
                    system.system_reset()

                    sign = -np.sign(grad_res-res)
                    new_Delta = ini_Delta*(1+step_len_ratio*sign*np.random.rand())

                    system.Delta = new_Delta
                    system.omega_s = system.cooperativity*kappa*Gamma / (4*system.Delta)
                    system.evolve_photons(Delta_ini_sequence)
                    new_res = system.evaluation(mode = target)[0]
                    system.system_reset()

                    if new_res<res:
                        T = T/(1+500/times/atom_number)
                        new_dres = res-new_res
                        res = deepcopy(new_res)
                        if res<min_res:
                            min_res = res*1.0
                        ini_Delta = new_Delta*1.0
                    else:
                        Tjudge = np.exp(-(new_res-res)/T)
                        random = np.random.rand()
                        if random<Tjudge:
                            res = deepcopy(new_res)
                            ini_Delta = new_Delta*1.0
                        else:
                            system.Delta = ini_Delta
                            system.omega_s = system.cooperativity*kappa*Gamma / (4*system.Delta)
                
                if new_dres>dres:
                    dres = new_dres*1.0

                print("\r", end="")
                print(f"Scan finished:{time+1}/{times},min_res = {min_res},Delta = {ini_Delta},res = {res}, dres = {dres}", end="")
                sys.stdout.flush()
            ini_photons[seq] = photons(ini_photons[seq].typ,ini_order,ini_delta,ini_t)
        if dres < res*10**(-5):
            T = T*(1+7.5/atom_number)
            
    final = sequence(ini_photons)
    for i in range(len(final.photons)):
        print(f"delta{i} = ",final.photons[i].delta)
        print(f"order{i} = ",final.photons[i].order)
        print(f"t{i} = ",final.photons[i].t)
        print(f"typ{i} = \""+final.photons[i].typ+"\"")
    print("Delta = ",ini_Delta)
    
    return final,ini_Delta

def fit_energy_level(target,Np,etamax,ini_order = np.array([None,None]),ini_delta = np.array([None,None]), times = 1000):
    N = len(target)-1
    T = 0.3
    ini_Delta =  -4115000000.0
    ini_eta = etamax
    system = cavity_atom_light_system(atom_number = N, cooperativity = ini_eta, Delta = ini_Delta, Omega = 0.2*10**6*2*np.pi)
    if (ini_order == None).all():
        ini_order = np.array([N/3+i*N/3/Np for i in range(Np)])
    if (ini_delta == None).all():
        ini_delta = np.array([np.min(target)/(i+1)/(N+1)*(1+ini_order[i]*etamax/(1+4*(ini_Delta+ini_order[i]*system.omega_s)**2/Gamma**2)) for i in range(Np)])
    ini_sequence = sequence([photons("x",ini_order,ini_delta,1.0e-6)])
    res = np.sum(((target-np.real(system.show_H_AC_photons(ini_sequence)[0]))[int(13*N/40):int(15*N/20)])**2)
    
    fig,ax = plt.subplots()
    ax.scatter(np.arange(N+1),target)
    ax.scatter(np.arange(N+1),np.real(system.show_H_AC_photons(ini_sequence)[0]))
    plt.show()
    
    grad_len_ratio = 10.0/times
    step_len_ratio = 100.0/times
    for time in range(times):
        dres = 0
        for order in range(Np):
            new_dres = 0
            
            new_order = ini_order*1.0
            grad_order = ini_order*1.0
            grad_delta = ini_delta*1.0
            grad_order[order] = grad_order[order]*(1+grad_len_ratio)
            grad_sequence = sequence([photons("x",grad_order,grad_delta,1.0e-6)])
            grad_res = np.sum(((target-np.real(system.show_H_AC_photons(grad_sequence)[0]))[int(13*N/40):int(15*N/20)])**2)
            sign = -np.sign(grad_res-res)
            new_order[order] = new_order[order]*(1+step_len_ratio*sign*np.random.rand())
            new_delta = ini_delta*1.0
            new_sequence = sequence([photons("x",new_order,new_delta,1.0e-6)])
            new_res = np.sum(((target-np.real(system.show_H_AC_photons(new_sequence)[0]))[int(13*N/40):int(15*N/20)])**2)
            
            if new_res<res:
                T = T/(1+100/times/N)
                new_dres = res-new_res
                res = deepcopy(new_res)
                ini_order = new_order*1.0
            else:
                Tjudge = np.exp(-(new_res-res)/T)
                random = np.random.rand()
                if random<Tjudge:
                    T = T/(1+100/times/N)
                    res = deepcopy(new_res)
                    ini_order = new_order*1.0
            if new_dres>dres:
                dres = new_dres*1.0
                
            new_order = ini_order*1.0
            grad_order = ini_order*1.0
            grad_delta = ini_delta*1.0
            grad_delta[order] = grad_delta[order]*(1+grad_len_ratio)
            grad_sequence = sequence([photons("x",grad_order,grad_delta,1.0e-6)])
            grad_res = np.sum(((target-np.real(system.show_H_AC_photons(grad_sequence)[0]))[int(13*N/40):int(15*N/20)])**2)
            sign = -np.sign(grad_res-res)
            new_delta[order] = new_delta[order]*(1+step_len_ratio*sign*np.random.rand())
            new_order = ini_order*1.0
            new_sequence = sequence([photons("x",new_order,new_delta,1.0e-6)])
            new_res = np.sum(((target-np.real(system.show_H_AC_photons(new_sequence)[0]))[int(13*N/40):int(15*N/20)])**2)
            
            if new_res<res:
                T = T/(1+100/times/N)
                new_dres = res-new_res
                res = deepcopy(new_res)
                ini_delta = new_delta*1.0
            else:
                Tjudge = np.exp(-(new_res-res)/T)
                random = np.random.rand()
                if random<Tjudge:
                    T = T/(1+100/times/N)
                    res = deepcopy(new_res)
                    ini_delta = new_delta*1.0
            if new_dres>dres:
                dres = new_dres*1.0
                
            print("\r", end="")
            print(f"Scan finished:{time+1}/{times}, res = {res}, dres = {dres}", end="")
            sys.stdout.flush()
        if dres < res*10**(-5):
            T = T*(1+25/N)
    
    ini_sequence = sequence([photons("x",ini_order,ini_delta,1.0e-6)])
    fig,ax = plt.subplots()
    ax.scatter(np.arange(N+1),target)
    ax.scatter(np.arange(N+1),np.real(system.show_H_AC_photons(ini_sequence)))
    plt.show()
    
    return photons("x",ini_order,ini_delta,1.0e-6)
    
        
def create_gif(image_list, gif_name, duration=0.05):
    frames = []

    for i in range(int(1/duration)):
        frames.append(imageio.imread(image_list[0]))

    for image_name in image_list:
        frames.append(imageio.imread(image_name))

    for i in range(int(1/duration)):
        frames.append(imageio.imread(image_list[-1]))
    imageio.mimsave(gif_name, frames, 'GIF', duration=duration)
    return None

Using matplotlib backend: Qt5Agg


In [11]:
delta0 =  np.array([0.])
order0 =  np.array([28.96868897])
t0 =  1.5155999580658109e-07
typ0 = "x"
delta1 =  np.array([-81031001.57593244])
order1 =  np.array([47.19472869])
t1 =  2.3468752534542159e-07
typ1 = "x"
Delta =  -5946171681.087355
atom_number = 50
ini_sequence = sequence(np.array([photons(typ0,order0,delta0,t0),photons(typ1,order1,delta1,t1)]))

system = cavity_atom_light_system(atom_number = 50, cooperativity = 1000, Delta = Delta, Omega = 0.2*10**6*2*np.pi)
system.evolve_photons(ini_sequence)
system.visualize()

target = "N1N1"
print(system.evaluation(mode = target))

H_AC_1p = system.show_H_AC_photons(ini_sequence)[1]


fig,ax = plt.subplots()
ax.scatter(np.arange(51),np.real(H_AC_1p)*10**(-6))
ax.set_xlabel("Dicke State Level")
ax.set_ylabel("Energy Shift/MHz")
ax.set_title("Energy level, One Light Generating W State")
plt.show()

(-0.9200017631770638, 0)


In [9]:
delta0 =  np.array([0.])
order0 =  np.array([3.00191979])
t0 =  4.1399943765870936e-07
typ0 = "x"
delta1 =  np.array([-1770840.01367247])
order1 =  np.array([32.74001811149749])
t1 =  2.1773138739050565e-06
typ1 = "x"
Delta =  -2*np.pi*3.4*10**9
atom_number = 50
ini_sequence = sequence(np.array([photons(typ0,order0,delta0,t0),photons(typ1,order1,delta1,t1)]))

system = cavity_atom_light_system(atom_number = 50, cooperativity = 1000, Delta = Delta, Omega = 0.2*10**6*2*np.pi)
system.evolve_photons(ini_sequence)
system.visualize()

H_AC_1p = system.show_H_AC_photons(ini_sequence)[1]



print(system.evaluation(mode = "N0N50"))

fig,ax = plt.subplots()
ax.scatter(np.arange(51),np.real(H_AC_1p)*10**(-6))
ax.set_xlabel("Dicke State Level")
ax.set_ylabel("Energy Shift/MHz")
ax.set_title("Energy level, One Light Generating W State")
plt.show()

None


In [27]:
# state |25>
# 1 step
fidelity_25_1 = 1-0.089841
t_25_1 = [1.49559822e-06]
diagonal_element_25_1 = [-3.16248744e+06-0.j, -2.81125658e+06-0.j, -1.88343174e+06-0.j,
  -6.69879533e+05-0.j, -7.58396596e-05-0.j, -4.49591986e-06-0.j,
  -3.61164788e-05-0.j, -3.47472034e+05-0.j, -5.66701086e+05-0.j,
  -9.17497154e-06-0.j, -1.18770360e+06-0.j, -4.24919444e+06-0.j,
  -2.56808490e+06-0.j, -1.71103536e-05-0.j, -4.74300315e+06-0.j,
  -4.82294165e+06-0.j, -1.08944703e-06-0.j, -7.81979461e+06-0.j,
  -1.61234811e+07-0.j, -1.52895419e+07-0.j, -1.69508950e-07-0.j,
  -5.64427380e-07-0.j, -2.79460806e+07-0.j, -1.36641023e+07-0.j,
  -5.46715395e+07-0.j, -7.59357675e+06-0.j, -4.80296590e+13-0.j,
  -2.28161118e+00-0.j, -6.20767979e+06-0.j, -1.20629731e+01-0.j,
  -6.67389606e+06-0.j, -8.51361493e+09-0.j, -4.42724177e+03-0.j,
  -5.69832751e+06-0.j, -7.66598746e+02-0.j, -4.54184675e+06-0.j,
  -8.20420248e+03-0.j, -5.33493439e+05-0.j, -1.63631602e+04-0.j,
  -1.47426538e+05-0.j, -2.47967110e+04-0.j, -1.40510525e+04-0.j,
  -2.68038299e+05-0.j, -1.52735698e+05-0.j, -1.99474318e+06-0.j,
  -7.57457527e+05-0.j, -1.05481349e+06-0.j, -1.00000000e+06-0.j,
  -1.00000000e+06-0.j, -1.00000000e+06-0.j, -1.00000000e+06-0.j]

fig,ax = plt.subplots()
ax.scatter(np.arange(51),np.real(diagonal_element_25_1))
plt.show()

fidelity25_2 = 1-0.040109
t_25_2 = [1.1673422e-06,4.9516496e-07]
diagonal_element_25_2 = [[-1.71030665e+06-0.j, -2.35134880e+06-0.j, -3.66814899e+06-0.j,
  -4.82262088e+06-0.j, -4.48493967e+06-0.j, -1.74718065e+06-0.j,
  -2.56563562e+05-0.j, -2.02985599e+06-0.j, -1.15673554e+06-0.j,
  -3.10954392e-02-0.j, -2.71344852e+06-0.j, -2.27583877e-01-0.j,
  -2.66425688e+03-0.j, -8.39236663e+06-0.j, -1.23462085e+00-0.j,
  -1.40385487e-01-0.j, -1.63667820e+07-0.j, -7.59138532e+06-0.j,
  -7.01760329e+00-0.j, -1.26935956e+07-0.j, -5.35440012e+06-0.j,
  -4.93032339e+06-0.j, -1.65397138e+07-0.j, -4.06796187e+06-0.j,
  -1.02666831e-03-0.j, -4.27543684e+08-0.j, -3.26825474e+05-0.j,
  -1.20183940e+06-0.j, -2.86100651e+05-0.j, -7.43701483e+06-0.j,
  -2.25984561e-01-0.j, -1.62325740e+07-0.j, -3.23146200e-01-0.j,
  -1.25144032e+07-0.j, -2.25385851e+00-0.j, -2.18313264e+06-0.j,
  -7.02502202e+02-0.j, -2.93226347e+01-0.j, -5.37800918e+07-0.j,
  -8.05708422e+01-0.j, -6.64752984e+06-0.j, -8.58746481e+02-0.j,
  -4.43940166e+06-0.j, -6.68265216e+04-0.j, -6.12570297e+06-0.j,
  -6.52304443e+05-0.j, -9.10782552e+05-0.j, -1.00000000e+06-0.j,
  -1.00000000e+06-0.j, -1.00000000e+06-0.j, -1.00000000e+06-0.j],
 [-3.83251070e+06-0.j, -2.87473225e-01-0.j, -6.02754260e+02-0.j,
  -7.75392354e+06-0.j, -5.56316869e-02-0.j, -5.42286354e+06-0.j,
  -3.92582833e+07-0.j, -1.09900933e-02-0.j, -1.92515342e+07-0.j,
  -1.93924154e+03-0.j, -1.03904178e+07-0.j, -5.90154294e+06-0.j,
  -1.18591255e+06-0.j, -1.57417719e+07-0.j, -3.02867836e+06-0.j,
  -5.63903230e+06-0.j, -1.97557764e+07-0.j, -5.17841286e+06-0.j,
  -2.20416896e-02-0.j, -1.27572834e+07-0.j, -2.48394172e-02-0.j,
  -1.21141982e-02-0.j, -2.95674298e+07-0.j, -1.45141956e+07-0.j,
  -5.50665085e+07-0.j, -8.61260094e+06-0.j, -3.08720573e+09-0.j,
  -5.08661058e-04-0.j, -6.88764936e+06-0.j, -2.93534154e-03-0.j,
  -4.67694016e+06-0.j, -5.74212231e-02-0.j, -2.64499558e+06-0.j,
  -1.29327578e-02-0.j, -4.90113029e+08-0.j, -6.53220384e-02-0.j,
  -3.49664188e+06-0.j, -2.00377560e-01-0.j, -1.66232936e+07-0.j,
  -8.71361555e-01-0.j, -9.56728131e+06-0.j, -1.06586935e+01-0.j,
  -6.04515371e+06-0.j, -6.81190473e+02-0.j, -5.00058428e+06-0.j,
  -6.18555186e+04-0.j, -7.06661471e+06-0.j, -1.00000000e+06-0.j,
  -1.00000000e+06-0.j, -1.00000000e+06-0.j, -1.00000000e+06-0.j]]

fig,ax = plt.subplots()
ax.scatter(np.arange(51),np.real(diagonal_element_25_2[0]))
ax.scatter(np.arange(51),np.real(diagonal_element_25_2[1]))
plt.show()


fidelity_25_3 = 1-0.007793
t_25_3 = [1.03181688e-06,3.17979239e-07,2.58272342e-07]
diagonal_element_25_3 = [[-5.56799319e+06-0.j, -5.16530005e+06-0.j, -4.21595406e+06-0.j,
  -3.20339303e+06-0.j, -2.60645477e+06-0.j, -2.49247062e+06-0.j,
  -2.63349926e+06-0.j, -2.64403201e+06-0.j, -2.27133117e+06-0.j,
  -1.47589768e+06-0.j, -4.23676229e+05-0.j, -2.81531417e-01-0.j,
  -2.09019415e+05-0.j, -1.15545093e-05-0.j, -9.18734232e+02-0.j,
  -1.70593708e+06-0.j, -3.91829080e+05-0.j, -6.79208738e+06-0.j,
  -9.88569106e+06-0.j, -1.15102955e+05-0.j, -1.78599540e+07-0.j,
  -1.71836479e-06-0.j, -3.21190414e-06-0.j, -1.29041712e+07-0.j,
  -3.15671102e+07-0.j, -2.65693048e+07-0.j, -2.47318641e+07-0.j,
  -2.36066660e+07-0.j, -1.81628398e+07-0.j, -7.35897993e-03-0.j,
  -7.84828011e+07-0.j, -5.43877563e-04-0.j, -7.44975523e+07-0.j,
  -8.52306150e-02-0.j, -1.15139490e+07-0.j, -2.72486035e+08-0.j,
  -5.02604226e+02-0.j, -5.92354536e+08-0.j, -1.80327741e+04-0.j,
  -6.48973319e+02-0.j, -6.81282316e+05-0.j, -1.07307512e+03-0.j,
  -1.55516669e+06-0.j, -1.70335446e+04-0.j, -4.90200171e+06-0.j,
  -1.11371055e+05-0.j, -3.01348744e+06-0.j, -9.49245757e+05-0.j,
  -1.00000000e+06-0.j, -1.00000000e+06-0.j, -1.00000000e+06-0.j],
 [-7.04675320e+05-0.j, -9.23607171e+03-0.j, -1.12054257e+08-0.j,
  -2.66498218e+02-0.j, -8.19990613e+06-0.j, -2.83638392e+07-0.j,
  -2.36968946e-04-0.j, -4.20381842e+07-0.j, -7.84287578e-05-0.j,
  -1.02473199e+07-0.j, -9.26096994e-02-0.j, -8.28881175e+06-0.j,
  -9.41653209e+06-0.j, -1.93756860e+07-0.j, -1.62990488e+07-0.j,
  -2.54776753e+07-0.j, -1.29462793e+07-0.j, -2.48667178e+07-0.j,
  -2.08137080e+07-0.j, -1.86505896e+07-0.j, -3.07304457e+07-0.j,
  -2.30180312e+07-0.j, -3.24040662e+06-0.j, -5.95808520e+05-0.j,
  -2.09574210e-06-0.j, -1.51091656e+07-0.j, -3.75189657e-06-0.j,
  -1.85745965e+07-0.j, -6.93378400e+05-0.j, -1.43324568e+08-0.j,
  -2.17825751e+04-0.j, -1.10304699e+07-0.j, -9.77324559e+08-0.j,
  -4.88639030e-04-0.j, -8.47468401e-02-0.j, -8.58280546e+06-0.j,
  -3.95145506e-02-0.j, -6.76046600e+07-0.j, -5.73834850e-01-0.j,
  -1.25916977e+07-0.j, -5.24297529e+00-0.j, -2.37166942e+07-0.j,
  -7.29504408e+02-0.j, -2.34009741e+03-0.j, -7.74391586e+04-0.j,
  -3.46143107e+04-0.j, -4.94697759e+06-0.j, -8.11165777e+05-0.j,
  -1.00000000e+06-0.j, -1.00000000e+06-0.j, -1.00000000e+06-0.j],
 [-1.00000000e+06-0.j, -1.66311631e+06-0.j, -4.70077470e+05-0.j,
  -1.96212600e+05-0.j, -3.96258497e+07-0.j, -5.33174257e+08-0.j,
  -1.18658847e+03-0.j, -1.61062138e+02-0.j, -8.06114535e-01-0.j,
  -2.75696220e+07-0.j, -1.87759297e-01-0.j, -1.30283931e+04-0.j,
  -1.48907779e+08-0.j, -8.68855531e+06-0.j, -1.76890608e+07-0.j,
  -8.14444159e+07-0.j, -4.56972544e-05-0.j, -2.34734711e+07-0.j,
  -2.84455856e-05-0.j, -6.17490048e+07-0.j, -5.04490206e+06-0.j,
  -7.66810405e+06-0.j, -8.89815276e+06-0.j, -3.47076682e+07-0.j,
  -5.47546679e+06-0.j, -2.42851232e+07-0.j, -1.78937734e+07-0.j,
  -8.23342018e-07-0.j, -4.43746523e+07-0.j, -1.59699358e+07-0.j,
  -1.18329558e+08-0.j, -1.65152483e+07-0.j, -2.42622707e+08-0.j,
  -8.30196473e-03-0.j, -2.50262388e+07-0.j, -1.84691161e-03-0.j,
  -2.09346670e+07-0.j, -2.54840268e-01-0.j, -2.42496584e+07-0.j,
  -2.11588835e+07-0.j, -1.78083828e+02-0.j, -7.46155725e+07-0.j,
  -7.89632163e+04-0.j, -1.48310905e+04-0.j, -6.69880380e+06-0.j,
  -4.70888866e+05-0.j, -1.00000000e+06-0.j, -1.00000000e+06-0.j,
  -1.00000000e+06-0.j, -1.00000000e+06-0.j, -1.00000000e+06-0.j]]

fig,ax = plt.subplots()
ax.scatter(np.arange(51),np.real(diagonal_element_25_3[0])*10**(-6),label = "step 1")
ax.scatter(np.arange(51),np.real(diagonal_element_25_3[1])*10**(-6),label = "step 2")
ax.scatter(np.arange(51),np.real(diagonal_element_25_3[2])*10**(-6),label = "step 3")
ax.set_xlabel("Dicke State Level")
ax.set_ylabel("Energy Shift/MHz")
ax.legend()
ax.set_title("Energy level, Generating Dicke 25 State")
plt.show()

In [16]:
# state |0> + |50>
fidelity_GHZ_1 = 1-0.003
t_GHZ_1 = [2.47730197e-06]
diagonal_element_GHZ_1 = [-1.56342088e+06-0.j, -1.53145752e+06-0.j, 
-1.29979231e+06-0.j, -8.60319359e+05-0.j, -5.35801998e+05-0.j, 
-5.52692444e+05-0.j, -6.07998001e+05-0.j, -5.08885712e+05-0.j, 
-5.75110646e+05-0.j, -6.88404594e+05-0.j, -4.49372982e+05-0.j, 
-3.31430986e+05-0.j, -4.12112181e+05-0.j, -2.08363114e+05-0.j, 
-4.28029904e+05-0.j, -8.28937700e+05-0.j, -5.23711039e+05-0.j, 
-9.16539148e+05-0.j, -1.46536002e+06-0.j, -7.83463038e+05-0.j, 
-1.49859851e+06-0.j, -1.56647523e+06-0.j, -5.39112804e-01-0.j, 
-5.18691463e+06-0.j, -5.29732202e-05-0.j, -2.14604316e+07-0.j, 
-9.62979812e-05-0.j, -8.82475720e+06-0.j, -7.04812442e+06-0.j, 
-3.25421638e+05-0.j, -7.42995310e+05-0.j, -3.06470481e+06-0.j, 
-3.00317469e+06-0.j, -2.33649118e+06-0.j, -1.82463793e+06-0.j, 
-1.94969226e+06-0.j, -2.61427987e+06-0.j, -3.07200991e+06-0.j, 
-3.15250586e+06-0.j, -3.02677721e+06-0.j, -2.78410176e+06-0.j, 
-2.48804940e+06-0.j, -2.33105438e+06-0.j, -2.40095696e+06-0.j, 
-2.41623428e+06-0.j, -2.21041303e+06-0.j, -1.85118413e+06-0.j, 
-1.51940903e+06-0.j, -1.35396918e+06-0.j, -1.35229146e+06-0.j, -1.40954249e+06-0.j]
fig,ax = plt.subplots()
ax.plot(np.arange(51),np.real(diagonal_element_GHZ_1))
plt.show()

In [19]:
# 10 20 30 40
fidelity_4d_2 = 1-0.097168
t_4d_2 = [1.55882956e-06, 3.05630628e-07]
diagonal_element_4d_2 = [[-3.13828022e+06-0.j, -2.08726167e+06-0.j, -4.73170714e+06-0.j,
  -2.01318164e+06-0.j, -1.81432549e+06-0.j, -8.63835120e+06-0.j,
  -1.08230051e+07-0.j, -6.63798038e+06-0.j, -2.50035536e-02-0.j,
  -7.75959975e+06-0.j, -1.69116174e+07-0.j, -2.06908167e+07-0.j,
  -1.97170543e+07-0.j, -1.58607431e+07-0.j, -9.45460722e+06-0.j,
  -1.45352630e+07-0.j, -2.35878643e+07-0.j, -1.11328272e+07-0.j,
  -2.46400218e+06-0.j, -1.57017466e+07-0.j, -3.48564415e-06-0.j,
  -1.29433288e-03-0.j, -1.54418365e+06-0.j, -6.81201681e+06-0.j,
  -2.41734430e+07-0.j, -3.14740337e-05-0.j, -4.55651281e+06-0.j,
  -6.81054248e-06-0.j, -9.13855158e-05-0.j, -7.11237527e+06-0.j,
  -2.60505829e+07-0.j, -1.99666501e+07-0.j, -2.54695019e+05-0.j,
  -1.24727409e+07-0.j, -2.48852145e+07-0.j, -5.13836464e+06-0.j,
  -2.98154280e-07-0.j, -3.44301689e+07-0.j, -3.86643984e+07-0.j,
  -1.42207448e-04-0.j, -8.26001819e-04-0.j, -1.33785613e+00-0.j,
  -3.75793490e+04-0.j, -5.34415382e+07-0.j, -1.79782320e+02-0.j,
  -8.30856776e+06-0.j, -5.61779560e+02-0.j, -1.10272263e+07-0.j,
  -3.17434080e+04-0.j, -4.07188219e+06-0.j, -3.55090921e+05-0.j],
 [-2.18850456e+04-0.j, -1.17998148e+13-0.j, -3.80575015e-02-0.j,
  -8.22221430e+13-0.j, -4.36236719e+07-0.j, -1.25073010e-05-0.j,
  -4.09898521e+07-0.j, -2.49947709e+07-0.j, -2.70014528e+07-0.j,
  -2.66937791e+07-0.j, -1.41800299e+07-0.j, -8.39830782e+06-0.j,
  -4.57444191e+07-0.j, -3.41060366e+06-0.j, -4.25339179e+06-0.j,
  -3.71085159e+07-0.j, -5.45689917e+04-0.j, -4.02489506e-01-0.j,
  -2.37512992e+07-0.j, -2.40135458e+06-0.j, -2.29078642e+07-0.j,
  -3.23529674e+07-0.j, -1.51499098e+07-0.j, -4.08260669e+07-0.j,
  -2.85424843e+07-0.j, -2.03819447e+06-0.j, -1.50491305e+07-0.j,
  -3.52764097e+07-0.j, -2.81335121e+07-0.j, -3.45971814e+06-0.j,
  -6.47258372e+06-0.j, -2.00018107e-06-0.j, -3.78713007e+07-0.j,
  -1.02139672e+07-0.j, -4.17965472e+07-0.j, -1.74597618e-06-0.j,
  -2.88803077e-06-0.j, -2.94054818e-06-0.j, -1.85543331e+06-0.j,
  -7.78830747e+06-0.j, -5.40022519e+06-0.j, -1.26181100e+11-0.j,
  -3.40173087e+00-0.j, -3.86805260e+07-0.j, -3.93404309e+13-0.j,
  -1.57219708e+03-0.j, -3.27052935e+06-0.j, -6.95886258e+03-0.j,
  -2.07710487e+05-0.j, -5.64480461e+05-0.j, -5.90834547e+05-0.j]]
fig,ax = plt.subplots()
ax.scatter(np.arange(51),np.real(diagonal_element_4d_2[0]))
ax.scatter(np.arange(51),np.real(diagonal_element_4d_2[1]))
plt.show()

In [22]:
# |W>
target = "N1N1"
fidelity_W_1 = 1e-8
t_W_1 = [3.53429767e-07]
diagonal_element_W_1 = [-1.62705976e+04-0.j, -1.03209865e+04-0.j, -11.71112435e+7-0.j,
  -1.58403566e+01-0.j, -4.05220696e+04-0.j, -6.67241820e+01-0.j,
  -5.01852541e+04-0.j, -2.13101901e+02-0.j, -1.39472807e+05-0.j,
  -6.03212194e+02-0.j, -6.60206567e+05-0.j, -2.03118900e+04-0.j,
  -3.42057633e+05-0.j, -2.04847522e+05-0.j, -8.06182838e+05-0.j,
  -7.00379868e+05-0.j, -6.82935940e+05-0.j, -6.82935940e+05-0.j,
  -6.82935940e+05-0.j, -6.82935940e+05-0.j, -6.82935940e+05-0.j,
  -6.82935940e+05-0.j, -6.82935940e+05-0.j, -6.82935940e+05-0.j,
  -6.82935940e+05-0.j, -6.82935940e+05-0.j, -6.82935940e+05-0.j,
  -6.82935940e+05-0.j, -6.82935940e+05-0.j, -6.82935940e+05-0.j,
  -6.82935940e+05-0.j, -6.82935940e+05-0.j, -6.82935940e+05-0.j,
  -6.82935940e+05-0.j, -6.82935940e+05-0.j, -6.82935940e+05-0.j,
  -6.82935940e+05-0.j, -6.82935940e+05-0.j, -6.82935940e+05-0.j,
  -6.82935940e+05-0.j, -6.82935940e+05-0.j, -6.82935940e+05-0.j,
  -6.82935940e+05-0.j, -6.82935940e+05-0.j, -6.82935940e+05-0.j,
  -6.82935940e+05-0.j, -6.82935940e+05-0.j, -6.82935940e+05-0.j,
  -6.82935940e+05-0.j, -6.82935940e+05-0.j, -6.82935940e+05-0.j]
fig,ax = plt.subplots()
ax.scatter(np.arange(51),np.real(diagonal_element_W_1))
ax.set_xlabel("Dicke State Level")
ax.set_ylabel("Energy Shift/MHz")
ax.set_title("Energy level, Generating W state")
plt.show()

In [7]:
N = 100
Omega = 0.2*10**6*2*np.pi
Delta = -4115000000.0

delta1 = np.array([0.])
order1 = np.array([12.22113492])
t1 = np.pi/3/Omega+np.pi/10/Omega*1
typ1 = "x"

ini_sequence = sequence(np.array([photons(typ1,order1,delta1,t1)]))


system = cavity_atom_light_system(atom_number = N, cooperativity = 200, Delta = Delta, Omega = Omega)
system.evolve_photons(ini_sequence)
system.visualize()

In [28]:
target_100 = np.array([-6.44736681e+03,-6.44736681e+03,-6.30448912e+03,-6.81594388e+03
,-5.83067003e+03,-6.56232370e+03,-6.37724352e+03,-5.37316000e+03
,-4.21650327e+03,-3.43876931e+03,-2.76957684e+03,-4.02424043e+03
,-4.50431368e+02,-2.31378412e+07,-3.25265748e+07,-3.40456987e+07
,-3.52853524e+07,-3.50887051e+07,-3.42926345e+07,-3.32654011e+07
,-3.23735022e+07,-3.06256303e+07,-2.91478117e+07,-2.77545066e+07
,-2.64885582e+07,-2.48065177e+07,-2.32516532e+07,-2.17645484e+07
,-1.97855682e+07,-1.74077292e+07,-1.46733828e+07,-1.16214545e+07
,-8.59378641e+06,-5.93747523e+06,-3.87852605e+06,-2.55216042e+06
,-1.94342547e+06,-1.84048872e+06,-2.09985689e+06,-2.68652535e+06
,-3.44939043e+06,-4.11525480e+06,-4.46564021e+06,-4.62452374e+06
,-5.12377254e+06,-6.35192931e+06,-8.38084121e+06,-1.11500817e+07
,-1.40498151e+07,-1.60900676e+07,-1.87208651e+07,-2.44416329e+07
,-3.25838489e+07,-4.02205155e+07,-4.78700115e+07,-5.53635865e+07
,-6.36823068e+07,-7.99693330e+07,-7.79393149e+07,-1.73604060e+08
,-1.84962472e+08,-2.49729167e+08,-3.11784120e+08,-3.61383796e+08
,-4.78145287e+08,-5.14934905e+08,-2.05955708e+08,-1.32390769e+08
,-9.88830676e+07,-2.07199268e+07,-5.71406811e+00,-2.06456268e+02
,-2.60085434e+08,-1.67621108e+02,-2.12543042e+07,-6.69058385e+02
,-3.72515037e+08,-4.13422309e+03,-1.33357429e+08,-1.97624824e+07
,-1.69084455e+05,-3.14774415e+07,-2.81818495e+08,-1.36579784e+06
,-4.99246824e+07,-3.36025461e+06,-4.09697117e+06,-4.11368829e+06
,-5.26695011e+06,-4.12098841e+06,-5.15936287e+06,-4.96260594e+06
,-4.83350471e+06,-4.83350471e+06,-4.83350471e+06,-4.83350471e+06
,-4.83350471e+06,-4.83350471e+06,-4.83350471e+06,-4.83350471e+06
,-4.83350471e+06])

t0 =  1.10594954e-06
t1 =  3.35119272e-07
ini_Delta =  -4115000000.0

system = cavity_atom_light_system(atom_number = 100, cooperativity = 200, Delta = ini_Delta, Omega = 0.2*10**6*2*np.pi)
opt_photons = [photons("x",np.array([3.0,3.0]),np.array([0,0]),t0),photons("x",np.array([41.05315533, 34.00417891, 42.67724553, 40.74924158, 41.02166873, 41.13946224,
 43.88325981, 45.76936813, 43.46935897, 40.99744805, 45.65502863, 45.54957476,
 77.12770892, 43.87915183, 48.7839165,  40.99086708, 34.00666379, 49.44890519,
 43.10832293, 51.40050657]),np.array([-3346399.85516118, -2486975.23956074, -5178452.60509671, -9890456.17483034,
 -7801888.28996133, -2558354.42541985, -3120265.48332675, -3437181.77888994,
 -3655438.13513873, -4898416.69426182, -2561599.8945809,   -524350.87329735,
   -23690.42670449, -2219101.62986116,  -998742.13963751, -3533589.53986137,
 -2937510.56261786,  -914358.97686151,  -924057.31484859, -1718110.43326022]),t1)]
ini_sequence = sequence(opt_photons)
print(ini_sequence.photons[1].delta[2])

fig,ax = plt.subplots()
ax.scatter(np.arange(100+1),target_100,label = f"Ideal Energy Wall")
ax.scatter(np.arange(100+1),np.real(system.show_H_AC_photons(sequence([opt_photons[1]]))),label = f"Fit with 20 frequencies")
ax.set_xlabel("Dicke State Level")
ax.set_ylabel("Energy Shift/Hz")
ax.legend()
ax.set_title("Energy level fit, 100 atoms 20 frequencies")
plt.show()

-5178452.60509671


In [29]:
target_50_4 =  np.array([-1.76572940e+07,-2.60542646e+07,-2.83642356e+07,-2.81519641e+07
,-2.75418035e+07,-2.66878641e+07,-2.53805814e+07,-2.47550691e+07
,-2.39569813e+07,-2.29593683e+07,-2.16082968e+07,-2.01056754e+07
,-1.84127137e+07,-1.65894844e+07,-1.48020932e+07,-1.33454804e+07
,-1.25358040e+07,-1.24590778e+07,-1.29532860e+07,-1.38991171e+07
,-1.48574333e+07,-1.55980381e+07,-1.69786909e+07,-1.96765049e+07
,-2.31086048e+07,-2.62158231e+07,-2.96343534e+07,-3.32173418e+07
,-3.93145120e+07,-4.40542966e+07,-5.65103245e+07,-6.44139139e+07
,-9.09637709e+07,-8.81142030e+07,-1.20628108e+08,-1.62676994e+08
,-2.49587320e+08,-3.51350636e+08,-4.37415453e+08,-4.39029139e+08
,-8.51325130e+08,-5.80589647e+08,-1.72858848e+08,-7.95802100e+07
,-6.05556869e+07,-2.92977920e+07,-4.75961873e+06,-4.59417242e+06
,-3.03412676e+06,-2.26086123e+06,-2.68773951e+06])

system = cavity_atom_light_system(atom_number = 50, cooperativity = 200, Delta = ini_Delta, Omega = 0.2*10**6*2*np.pi)
opt_photons = [photons("x",np.array([3.0,3.0]),np.array([0,0]),t0),photons("x",np.array([18.554715450180872, 15.104600558004833, 
                                                                                         20.689409279674813, 23.252873480612777]),
                                                                           np.array([-4587645.096951035, -128899430.37283333,
                                                                                     -6844411.479208152, -3703577.152291897]),t1)]
ini_sequence = sequence(opt_photons)
print(ini_sequence.photons[1].delta[2])

fig,ax = plt.subplots()
ax.scatter(np.arange(50+1),target_50,label = f"Ideal Energy Wall")
ax.scatter(np.arange(50+1),np.real(system.show_H_AC_photons(sequence([opt_photons[1]]))),label = f"Fit with 4 frequencies")
ax.set_xlabel("Dicke State Level")
ax.set_ylabel("Energy Shift/Hz")
ax.legend()
ax.set_title("Energy level fit, 50 atoms 4 frequencies")
plt.show()


-6844411.479208152


In [30]:
target_50 =  np.array([-1.76572940e+07,-2.60542646e+07,-2.83642356e+07,-2.81519641e+07
,-2.75418035e+07,-2.66878641e+07,-2.53805814e+07,-2.47550691e+07
,-2.39569813e+07,-2.29593683e+07,-2.16082968e+07,-2.01056754e+07
,-1.84127137e+07,-1.65894844e+07,-1.48020932e+07,-1.33454804e+07
,-1.25358040e+07,-1.24590778e+07,-1.29532860e+07,-1.38991171e+07
,-1.48574333e+07,-1.55980381e+07,-1.69786909e+07,-1.96765049e+07
,-2.31086048e+07,-2.62158231e+07,-2.96343534e+07,-3.32173418e+07
,-3.93145120e+07,-4.40542966e+07,-5.65103245e+07,-6.44139139e+07
,-9.09637709e+07,-8.81142030e+07,-1.20628108e+08,-1.62676994e+08
,-2.49587320e+08,-3.51350636e+08,-4.37415453e+08,-4.39029139e+08
,-8.51325130e+08,-5.80589647e+08,-1.72858848e+08,-7.95802100e+07
,-6.05556869e+07,-2.92977920e+07,-4.75961873e+06,-4.59417242e+06
,-3.03412676e+06,-2.26086123e+06,-2.68773951e+06])

system = cavity_atom_light_system(atom_number = 50, cooperativity = 200, Delta = ini_Delta, Omega = 0.2*10**6*2*np.pi)
opt_photons = [photons("x",np.array([3.0,3.0]),np.array([0,0]),t0),photons("x",np.array([16.66666667,17.15840561,16.931264,16.67585169,18.77991608,20.52627194
                                                                            ,20.80101247,20.80113886,24.24774026,22.87298035,20.71376355,25.59269096
                                                                            ,22.40642895,26.99042985,27.82458784,28.87238346,33.18709803,35.03870993
                                                                            ,30.61720512,32.03083845]),
                                                                            np.array([-20719436.3400608,-8903284.97445361,-7022894.60857448
                                                                            ,-6119853.09448543,-6881589.99877056,-2140171.35386113
                                                                            ,-2510397.35932708,-2956227.13461667,-2805384.98102558
                                                                            ,-1729585.14649017,-1489096.24748984,-2125605.20267164
                                                                            ,-2387970.37127643,-1981572.4444592,-1323639.53665706
                                                                            ,-1422958.58577755,-1355892.67575984,-3405474.40760201
                                                                            ,-1455490.22224637,-1454010.35806351]),t1)]
ini_sequence = sequence(opt_photons)
print(ini_sequence.photons[1].delta[2])

fig,ax = plt.subplots()
ax.scatter(np.arange(50+1),target_50,label = f"Ideal Energy Wall")
ax.scatter(np.arange(50+1),np.real(system.show_H_AC_photons(sequence([opt_photons[1]]))),label = f"Fit with 20 frequencies")
ax.set_xlabel("Dicke State Level")
ax.set_ylabel("Energy Shift/Hz")
ax.legend()
ax.set_title("Energy level fit, 50 atoms 20 frequencies")
plt.show()



-7022894.60857448


In [31]:
target_100 = np.array([-6.44736681e+03,-6.44736681e+03,-6.30448912e+03,-6.81594388e+03
,-5.83067003e+03,-6.56232370e+03,-6.37724352e+03,-5.37316000e+03
,-4.21650327e+03,-3.43876931e+03,-2.76957684e+03,-4.02424043e+03
,-4.50431368e+02,-2.31378412e+07,-3.25265748e+07,-3.40456987e+07
,-3.52853524e+07,-3.50887051e+07,-3.42926345e+07,-3.32654011e+07
,-3.23735022e+07,-3.06256303e+07,-2.91478117e+07,-2.77545066e+07
,-2.64885582e+07,-2.48065177e+07,-2.32516532e+07,-2.17645484e+07
,-1.97855682e+07,-1.74077292e+07,-1.46733828e+07,-1.16214545e+07
,-8.59378641e+06,-5.93747523e+06,-3.87852605e+06,-2.55216042e+06
,-1.94342547e+06,-1.84048872e+06,-2.09985689e+06,-2.68652535e+06
,-3.44939043e+06,-4.11525480e+06,-4.46564021e+06,-4.62452374e+06
,-5.12377254e+06,-6.35192931e+06,-8.38084121e+06,-1.11500817e+07
,-1.40498151e+07,-1.60900676e+07,-1.87208651e+07,-2.44416329e+07
,-3.25838489e+07,-4.02205155e+07,-4.78700115e+07,-5.53635865e+07
,-6.36823068e+07,-7.99693330e+07,-7.79393149e+07,-1.73604060e+08
,-1.84962472e+08,-2.49729167e+08,-3.11784120e+08,-3.61383796e+08
,-4.78145287e+08,-5.14934905e+08,-2.05955708e+08,-1.32390769e+08
,-9.88830676e+07,-2.07199268e+07,-5.71406811e+00,-2.06456268e+02
,-2.60085434e+08,-1.67621108e+02,-2.12543042e+07,-6.69058385e+02
,-3.72515037e+08,-4.13422309e+03,-1.33357429e+08,-1.97624824e+07
,-1.69084455e+05,-3.14774415e+07,-2.81818495e+08,-1.36579784e+06
,-4.99246824e+07,-3.36025461e+06,-4.09697117e+06,-4.11368829e+06
,-5.26695011e+06,-4.12098841e+06,-5.15936287e+06,-4.96260594e+06
,-4.83350471e+06,-4.83350471e+06,-4.83350471e+06,-4.83350471e+06
,-4.83350471e+06,-4.83350471e+06,-4.83350471e+06,-4.83350471e+06
,-4.83350471e+06])

t0 =  1.10594954e-06
t1 =  3.35119272e-07
ini_Delta =  -4115000000.0

system = cavity_atom_light_system(atom_number = 100, cooperativity = 200, Delta = ini_Delta, Omega = 0.2*10**6*2*np.pi)
opt_photons = [photons("x",np.array([3.0,3.0]),np.array([0,0]),t0),photons("x",np.array([26.389377631350072, 41.064199615852594,
                                                                                         43.77932133984563, 40.64437820358573]),
                                                                           np.array([-31801891.874933723, -25979831.04587118, 
                                                                                     -15672233.768405149, -7992678.205318747]),t1)]
ini_sequence = sequence(opt_photons)
print(ini_sequence.photons[1].delta[2])

fig,ax = plt.subplots()
ax.scatter(np.arange(100+1),target_100,label = f"Ideal Energy Wall")
ax.scatter(np.arange(100+1),np.real(system.show_H_AC_photons(sequence([opt_photons[1]]))),label = f"Fit with 4 frequencies")
ax.set_xlabel("Dicke State Level")
ax.set_ylabel("Energy Shift/Hz")
ax.legend()
ax.set_title("Energy level fit, 100 atoms 4 frequencies")
plt.show()

-15672233.768405149


In [3]:
Xrs = [0.07803891, 0.07559933, 0.07316742, 0.0709256,  0.06898203, 0.06714768, 0.0653255,  0.06356519,
       0.06192809, 0.06042983, 0.05900193, 0.05761168,
       0.05627765, 0.05501737, 0.05383157, 0.05269455, 0.05161035, 0.05056779,
       0.04956127, 0.04860252, 0.04767348, 0.04678798, 0.04594495, 0.04512891,
       0.04434065, 0.04356877]
Xrs_Loss_150 = np.array([0.1288271 , 0.12503938, 0.12150618, 0.11815393, 0.11505763,
       0.11214922, 0.10938308, 0.106756  , 0.10427494, 0.10192337,
       0.09969098, 0.09755526, 0.09552749, 0.09361392, 0.09178552,
       0.09003638, 0.08835301, 0.08674286, 0.08521512, 0.08372325,
       0.08230292, 0.08094071, 0.07962379, 0.07835341, 0.07712329,
       0.07593828])
Xrs_Loss_150_N200 = np.array([0.12888771, 0.12323929, 0.11825511, 0.11354562, 0.10929115,
       0.10549344, 0.10185851, 0.09863182, 0.0957518 , 0.09276338,
       0.09000137, 0.08744802, 0.08509719, 0.08289584, 0.08077201,
       0.07879671, 0.07696925, 0.07522962, 0.07352998, 0.0719472 ,
       0.07045084, 0.06905372, 0.06766698, 0.06634342, 0.06512286,
       0.06393773, 0.0628158 , 0.06169142, 0.06061958, 0.05959135,
       0.05861085, 0.05766581, 0.05677934, 0.05590159, 0.05508372,
       0.05426846, 0.05347263, 0.05271526, 0.05198944, 0.05127572,
       0.05060197, 0.04994295, 0.04929178, 0.04863696, 0.04804381,
       0.04746056, 0.04689318, 0.04631983, 0.04578204, 0.04526153,
       0.0447448 ])
Xrs_Loss_200 = np.array([0.12438353, 0.11990934, 0.11575324, 0.1118897 , 0.10860876,
       0.1056397 , 0.10288566, 0.1003567 , 0.09799141, 0.09578347,
       0.0937019 , 0.09172934, 0.08980824, 0.08801739, 0.08630663,
       0.08467571, 0.08308837, 0.08157617, 0.08012226, 0.07874403,
       0.07737577, 0.07608749, 0.07483827, 0.07364518, 0.07249427,
       0.07139599])
Xrs_scan = np.array([0.07760955, 0.0751425 , 0.07282821, 0.07066177, 0.06865799,
       0.06672961, 0.06493887, 0.06322896, 0.06161687, 0.06008036,
       0.05862703, 0.05726709, 0.05594056, 0.05470001, 0.05350526,
       0.05237512, 0.05128845, 0.05025237, 0.049246  , 0.04829618,
       0.04738111, 0.04649987, 0.04565367, 0.04483425, 0.04406067,
       0.04329619])
Xrs_Loss_scan = np.array([0.17202467, 0.16820156, 0.16487766, 0.16162623, 0.15854766,
       0.1555961 , 0.15288324, 0.15030358, 0.14774334, 0.14543072,
       0.14321711, 0.1410486 , 0.13901479, 0.13707639, 0.13519732,
       0.1334069 , 0.13174311, 0.13009841, 0.12857183, 0.12708589,
       0.12559733, 0.12424156, 0.12288201, 0.12158615, 0.12034465,
       0.11913633])

fig,ax = plt.subplots()
ax.scatter(np.array([50+i*2 for i in range(26)]),Xrs, label = "Xr genetic, no loss")
ax.scatter(np.array([50+i*2 for i in range(26)]),Xrs_scan[0:26], label = "Xr scan, no loss")
ax.scatter(np.array([50+i*2 for i in range(26)]),Xrs_Loss_150, label = "Xr genetic,eta = 150, with loss")
ax.scatter(np.array([50+i*2 for i in range(26)]),Xrs_Loss_200, label = "Xr genetic,eta = 200, with loss")
ax.scatter(np.array([50+i*2 for i in range(26)]),Xrs_Loss_scan, label = "Xr scan, with loss")
ax.set_xlabel("N")
ax.set_ylabel("Xr")
ax.set_title(f"Single frequecy N(50-100)-Xr, genetic evolution vs scan optimization")
ax.legend()
plt.show()


fitgn = np.polyfit(np.array([np.log(50+i*2) for i in range(26)]),np.log(Xrs),1)
fitsn = np.polyfit(np.array([np.log(50+i*2) for i in range(26)]),np.log(Xrs_scan),1)
fitg150 = np.polyfit(np.array([np.log(50+i*2) for i in range(26)]),np.log(Xrs_Loss_150),1)
fitg200 = np.polyfit(np.array([np.log(50+i*2) for i in range(26)]),np.log(Xrs_Loss_200),1)
fits = np.polyfit(np.array([np.log(50+i*2) for i in range(26)]),np.log(Xrs_Loss_scan),1)

print("genetic, no loss:",fitgn)
print("genetic,eta = 150, with loss:",fitg150)
print("genetic,eta = 200, with loss:",fitg200)
print("scan, no loss:",fitsn)
print("scan, with loss:",fits)


fig,ax = plt.subplots()
ax.scatter(np.array([np.log(50+i*2) for i in range(26)]),np.log(Xrs),label = f"genetic, no loss,scale as {round(fitgn[0],2)}")
ax.scatter(np.array([np.log(50+i*2) for i in range(26)]),np.log(Xrs_Loss_150),label = f"genetic,eta = 150, with loss,scale as {round(fitg150[0],2)}")
ax.scatter(np.array([np.log(50+i*2) for i in range(26)]),np.log(Xrs_Loss_200),label = f"genetic,eta = 200, with loss,scale as {round(fitg200[0],2)}")
ax.scatter(np.array([np.log(50+i*2) for i in range(26)]),np.log(Xrs_scan),label = f"scan, no loss,scale as {round(fitsn[0],2)}")
ax.scatter(np.array([np.log(50+i*2) for i in range(26)]),np.log(Xrs_Loss_scan),label = f"scan, with loss,scale as {round(fits[0],2)}")

ax.set_xlabel("Ln(N)")
ax.set_ylabel("Ln(Xr)")
ax.set_title(f"Single frequecy N(50-100)-Xr, genetic evolution vs scan optimization")
ax.legend()
plt.show()

genetic, no loss: [-0.84087503  0.74077048]
genetic,eta = 150, with loss: [-0.76257449  0.93407581]
genetic,eta = 200, with loss: [-0.78474608  0.97043653]
scan, no loss: [-0.84299232  0.74400719]
scan, with loss: [-0.52967745  0.30855113]


In [26]:
min_res_13_200=np.array([0.068282,0.065898,0.063934,0.062060,0.060781,0.058682,0.057236,0.055984,0.054316,
0.052714,0.050963,0.049533,0.048540,0.047804,0.046642,0.045898,0.045080,0.044207,0.043453,0.041857,
0.041009,0.040460,0.039724,0.038961,0.038296,0.037739])

min_res_08_200=np.array([0.070316,0.068645,0.066351,0.064280,0.062170,0.059825,0.058175,0.056701,0.054797,
0.053701,0.052591,0.051844,0.050925,0.050299,0.049662,0.046974,0.046049,0.045102,0.044413,0.043483,
0.042782,0.042396,0.041539,0.040719,0.040064,0.039718])

min_res_04_200=np.array([0.072674,0.070606,0.068260,0.066116,0.064325,0.062828,0.061012,0.059387,0.057705,
0.056231,0.054866,0.053678,0.052293,0.051035,0.049840,0.048720,0.047725,0.047017,0.046019,0.045167,
0.044247,0.043434,0.042709,0.041906,0.041137,0.040452])

Xrs_perf_50_100 = np.array([0.06543183, 0.06333629, 0.06135861, 0.05964955, 0.05801641,
       0.05646115, 0.05467115, 0.05338125, 0.0517691 , 0.05016819,
       0.04874031, 0.04761858, 0.04651163, 0.04566798, 0.04485527,
       0.04420728, 0.04313875, 0.04179013, 0.04067507, 0.03965227,
       0.03875317, 0.03790891, 0.03707256, 0.03633115, 0.03569463,
       0.03502875])

fitperfn = np.polyfit(np.array([np.log(50+i*2) for i in range(26)]),np.log(Xrs_perf_50_100),1)

fitmulti_13_200 = np.polyfit(np.array([np.log(50+i*2) for i in range(26)]),np.log(min_res_13_200),1)
fitmulti_08_200 = np.polyfit(np.array([np.log(50+i*2) for i in range(26)]),np.log(min_res_08_200),1)
fitmulti_04_200 = np.polyfit(np.array([np.log(50+i*2) for i in range(26)]),np.log(min_res_04_200),1)

Xrs_scan = np.array([0.07760955, 0.0751425 , 0.07282821, 0.07066177, 0.06865799,
       0.06672961, 0.06493887, 0.06322896, 0.06161687, 0.06008036,
       0.05862703, 0.05726709, 0.05594056, 0.05470001, 0.05350526,
       0.05237512, 0.05128845, 0.05025237, 0.049246  , 0.04829618,
       0.04738111, 0.04649987, 0.04565367, 0.04483425, 0.04406067,
       0.04329619])



fits = np.polyfit(np.array([np.log(50+i*2) for i in range(26)]),np.log(Xrs_scan),1)

fig,ax = plt.subplots()
ax.scatter(np.array([np.log(50+i*2) for i in range(26)]),np.log(min_res_13_200),label = f"13 Frequency Scaned data")
ax.scatter(np.array([np.log(50+i*2) for i in range(26)]),np.log(min_res_08_200),label = f"8 Frequency Scaned data")
ax.scatter(np.array([np.log(50+i*2) for i in range(26)]),np.log(min_res_04_200),label = f"4 Frequency Scaned data")
# ax.scatter(np.array([np.log(50+i*2) for i in range(26)]),np.array([np.log(50+i*2)*fitmulti_13_200[0]+fitmulti_13_200[1] for i in range(26)]),label = f"Fitted Multi 13 data")
ax.scatter(np.array([np.log(50+i*2) for i in range(26)]),np.log(Xrs_scan),label = f"Single Frequency Scaned data")
# ax.scatter(np.array([np.log(50+i*2) for i in range(26)]),np.array([np.log(50+i*2)*fits[0]+fits[1] for i in range(26)]),label = f"Fitted Single data")
ax.scatter(np.array([np.log(50+i*2) for i in range(26)]),np.log(Xrs_perf_50_100),label = f"Energy level limited data")

print(fitmulti_13_200[0])

ax.set_xlabel("Ln(N)")
ax.set_ylabel("Ln(Xr)")
ax.set_title(f"Multi VS Single N(50-100)-Xr with loss, Multi scale as {round(fitmulti_13_200[0],4)}")
ax.legend()

plt.show()

-0.8622220842250636


In [82]:
delta1 = np.array([0.,])
order1 = np.array([12.22113492])
t1 = 9.89201427859632e-07
typ1 = "x"
delta2 = np.array([-29863057.11682639])
order2 = np.array([21.91341307])
# t2 = 3.7635440192982807e-07
t2 = 4.034937269004413e-07
typ2 = "x"
# delta3 = np.array([0.,])
# order3 = np.array([12.22113492])
# t3 = 0.8e-06
# typ3 = "z"
# delta4 = np.array([0.,])
# order4 = np.array([12.22113492])
# t4 = 1/2*np.pi/(0.2*10**6*2*np.pi)*1.05
# typ4 = "x"
# delta5 = np.array([0.,])
# order5 = np.array([12.22113492])
# t5 = 1/2*np.pi/(0.2*10**6*2*np.pi)*0.95
# typ5 = "z"

Delta = -1929094742.73976

atom_number = 50

ini_sequence = sequence(np.array([photons(typ1,order1,delta1,t1),photons(typ2,order2,delta2,t2)]))
system = cavity_atom_light_system(atom_number = atom_number, cooperativity = 200, Delta = Delta, Omega = 0.2*10**6*2*np.pi)
system.evolve_photons(ini_sequence)
# system.evolve_photons(ini_sequence)
system.visualize()

In [83]:
# delta1 = np.array([0.,])
# order1 = np.array([12.22113492])
# t1 = 1.1071768611339502e-06
# typ1 = "x"
# delta2 = np.array([-2.02474787e+08, -1.90396446e+08, -1.71270940e+05, -5.83979153e+06])
# order2 = np.array([33.98787473,33.87177394,48.52182782,74.52239624])
# t2 = 4.5635440192982807e-07
# typ2 = "x"

# Delta = -4115000000.0

# atom_number = 100

delta1 = np.array([0.,])
order1 = np.array([12.22113492])
t1 = 9.89201427859632e-07
typ1 = "x"
delta2 = np.array([-29863057.11682639])
order2 = np.array([21.91341307])
# t2 = 3.7635440192982807e-07
t2 = 4.034937269004413e-07
typ2 = "x"

Delta = -1929094742.73976

atom_number = 50

ini_sequence = sequence(np.array([photons(typ1,order1,delta1,t1),photons(typ2,order2,delta2,t2)]))

animate_system = cavity_atom_light_system(atom_number = atom_number, cooperativity = 200, Delta = Delta, Omega = 0.2*10**6*2*np.pi)
ini_sequence = sequence(np.array([photons(typ1,order1,delta1,t1),photons(typ2,order2,delta2,t2)]))
# animate_system = cavity_atom_light_system(atom_number = 100, cooperativity = 56, Delta = -197*Gamma, Omega = 0.2*10**6*2*np.pi)
# animate_para1 = [0.0e-06,0.0e-06,1.593302185297957e-06,74,-423699254.15045,0.0e-06]
# animate_para2 = [0.0e-06,0.0e-06,0.0e-06,74,-423699254.15045,2.4134381601596233e-06]

time_step = 0.2*10**(-7)

num_frames = int((t1+t2)/time_step+1)

# facecolors list
colors = []

for frame in range(num_frames):
    t = frame*time_step
    
    colors.append(animate_system.evolve_photons_t(ini_sequence,t))
    animate_system.system_reset()
    print("\r", end="")
    print(f"frame generated:{frame}/{num_frames}", end="")
    sys.stdout.flush()


frame generated:69/70

In [84]:
theta = np.linspace(0, np.pi, 100)
phi = np.linspace(0, np.pi*4/2, 100)
theta_grid, phi_grid = np.meshgrid(theta, phi)
y = np.sin(theta_grid) * np.cos(phi_grid)
x = -np.sin(theta_grid) * np.sin(phi_grid)
z = -np.cos(theta_grid)

image_list = []
for i in range(num_frames):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    q_func = colors[i]
    norm = Normalize(vmin=np.min(q_func), vmax=np.max(q_func))
    
    ax.plot_surface(x, y, z, rstride=1, cstride=1, facecolors=plt.cm.jet(norm(q_func)), alpha=0.3, linewidth=0, antialiased=False)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('Husimi-Q Representation')

    plt.savefig(f"fig/20230913_Tearing_gif{i}.jpg")
    plt.close()
    image_list.append(f"fig/20230913_Tearing_gif{i}.jpg")
    print("\r", end="")
    print(f"data generated:{i}/{num_frames}", end="")
    sys.stdout.flush()
    
gif_name = "fig/20230913_Tearing_gif.gif"
duration = 0.05
create_gif(image_list, gif_name,duration)

data generated:69/70

In [115]:
# delta1 = np.array([0.,])
# order1 = np.array([12.22113492])
# t1 = 1.1071768611339502e-06
# typ1 = "x"
delta2 = np.array([-2.02474787e+08, -1.90396446e+08, -1.71270940e+05, -5.83979153e+06])
order2 = np.array([33.98787473,33.87177394,48.52182782,74.52239624])
t2 = 20.5635440192982807e-06
typ2 = "x"

Delta = -4115000000.0

atom_number = 100

animate_system = cavity_atom_light_system(atom_number = 100, cooperativity = 200, Delta = Delta, Omega = 0.2*10**6*2*np.pi)
ini_sequence = sequence(np.array([photons(typ2,order2,delta2,t2)]))
# animate_system = cavity_atom_light_system(atom_number = 100, cooperativity = 56, Delta = -197*Gamma, Omega = 0.2*10**6*2*np.pi)
# animate_para1 = [0.0e-06,0.0e-06,1.593302185297957e-06,74,-423699254.15045,0.0e-06]
# animate_para2 = [0.0e-06,0.0e-06,0.0e-06,74,-423699254.15045,2.4134381601596233e-06]


time_step = 2*10**(-7)

num_frames = int(t2/time_step+1)

# facecolors list
colors = []

for frame in range(num_frames):
    t = frame*time_step
    animate_system.psi = expm(-1j*(animate_system.S_y)*np.pi/100)@ animate_system.psi
    colors.append(animate_system.evolve_photons_t(ini_sequence,t))
    animate_system.system_reset()
    print("\r", end="")
    print(f"frame generated:{frame}/{num_frames}", end="")
    sys.stdout.flush()


frame generated:102/103

In [116]:
theta = np.linspace(0, np.pi, 100)
phi = np.linspace(0, np.pi*4/2, 100)
theta_grid, phi_grid = np.meshgrid(theta, phi)
y = np.sin(theta_grid) * np.cos(phi_grid)
x = -np.sin(theta_grid) * np.sin(phi_grid)
z = -np.cos(theta_grid)

image_list = []
for i in range(num_frames):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    q_func = colors[i]
    norm = Normalize(vmin=np.min(q_func), vmax=np.max(q_func))
    
    ax.plot_surface(x, y, z, rstride=1, cstride=1, facecolors=plt.cm.jet(norm(q_func)), alpha=0.3, linewidth=0, antialiased=False)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('Husimi-Q Representation')

    plt.savefig(f"fig/20230908_multicircle_ydetune{i}.jpg")
    plt.close()
    image_list.append(f"fig/20230908_multicircle_ydetune{i}.jpg")
    print("\r", end="")
    print(f"data generated:{i}/{num_frames}", end="")
    sys.stdout.flush()
    
gif_name = "fig/20230909_multicircle_ydetune.gif"
duration = 0.05
create_gif(image_list, gif_name,duration)

data generated:102/103

In [16]:
delta0 =  np.array([0.])
order0 =  np.array([6.24470702])
t0 =  1.0906036676675656e-06
typ0 = "x"
delta1 =  np.array([-55467253.80633463, -14243724.71063591, -16368831.77158581,
  -6269737.98514381,          -1760991.7125886,
  -1286549.07667044])
order1 =  np.array([42.27324073, 25.60576433, 35.25966267, 32.32134914, 48.32949331,
 50.39593893])
t1 =  3.6333386154208985e-07

typ1 = "x"
Delta =  -4148337336.9480805

ini_sequence = sequence(np.array([photons(typ0,order0,delta0,t0),photons(typ1,order1,delta1,t1)]))

atom_number = 100

animate_system = cavity_atom_light_system(atom_number = atom_number, cooperativity = 200, Delta = Delta, Omega = 0.2*10**6*2*np.pi)

# animate_system = cavity_atom_light_system(atom_number = 100, cooperativity = 56, Delta = -197*Gamma, Omega = 0.2*10**6*2*np.pi)
# animate_para1 = [0.0e-06,0.0e-06,1.593302185297957e-06,74,-423699254.15045,0.0e-06]
# animate_para2 = [0.0e-06,0.0e-06,0.0e-06,74,-423699254.15045,2.4134381601596233e-06]

time_step = 0.5*10**(-7)

num_frames = int((t0+t1)/time_step+1)

# facecolors list
colors = []
states = []

for frame in range(num_frames):
    t = frame*time_step
    colors.append(animate_system.evolve_photons_t(ini_sequence,t))
    states.append(animate_system.psi)
    animate_system.system_reset()
    print("\r", end="")
    print(f"frame generated:{frame}/{num_frames}", end="")
    sys.stdout.flush()

frame generated:29/30

In [25]:
# int(t0/time_step)
print(list(states)[25])

[[ 1.17894125e-15+3.76637117e-16j]
 [ 2.74721726e-15-8.65028857e-15j]
 [-5.93947128e-14-1.91948077e-14j]
 [-1.11652770e-13+3.38720523e-13j]
 [ 1.66456153e-12+5.59855592e-13j]
 [ 2.49892983e-12-7.27849200e-12j]
 [-2.88955496e-11-1.01312462e-11j]
 [-3.78395243e-11+1.05632294e-10j]
 [ 3.59232334e-10+1.31536377e-10j]
 [ 4.28912589e-10-1.14541172e-09j]
 [-3.44523625e-09-1.32004318e-09j]
 [-3.85358901e-09+9.82421107e-09j]
 [ 2.66664595e-08+1.07147096e-08j]
 [ 2.84723948e-08-6.91352703e-08j]
 [-1.71692789e-07-7.25206905e-08j]
 [-1.77493132e-07+4.09443793e-07j]
 [ 9.39620988e-07+4.18336936e-07j]
 [ 9.51308507e-07-2.07890264e-06j]
 [-4.44166550e-06-2.09071460e-06j]
 [-4.44725454e-06+9.17721522e-06j]
 [ 1.83603325e-05+9.16830831e-06j]
 [ 1.83401566e-05-3.56076786e-05j]
 [-6.70092391e-05-3.56368040e-05j]
 [-6.73281870e-05+1.22472803e-04j]
 [ 2.17571058e-04+1.23788120e-04j]
 [ 2.21662228e-04-3.75942960e-04j]
 [-6.32219983e-04-3.86857585e-04j]
 [-6.58484611e-04+1.03531183e-03j]
 [ 1.65168982e-03+1.

In [12]:
theta = np.linspace(0, np.pi, 100)
phi = np.linspace(0, np.pi*4/2, 100)
theta_grid, phi_grid = np.meshgrid(theta, phi)
y = np.sin(theta_grid) * np.cos(phi_grid)
x = -np.sin(theta_grid) * np.sin(phi_grid)
z = -np.cos(theta_grid)

image_list = []
for i in range(num_frames):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    q_func = colors[i]
    norm = Normalize(vmin=np.min(q_func), vmax=np.max(q_func))
    
    ax.plot_surface(x, y, z, rstride=1, cstride=1, facecolors=plt.cm.jet(norm(q_func)), alpha=0.3, linewidth=0, antialiased=False)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('Husimi-Q Representation')

    plt.savefig(f"fig/20231112_SqueVisua{i}.jpg")
    plt.close()
    image_list.append(f"fig/20231112_SqueVisua{i}.jpg")
    print("\r", end="")
    print(f"data generated:{i}/{num_frames}", end="")
    sys.stdout.flush()
    
gif_name = "fig/20231112_SqueVisua.gif"
duration = 0.05
create_gif(image_list, gif_name,duration)

data generated:29/30

In [4]:
delta0 =  np.array([0.])
order0 =  np.array([3.00191979])
t0 =  4.1399943765870936e-07
typ0 = "x"
delta1 =  np.array([-1770840.01367247])
order1 =  np.array([32.74001811149749])
t1 =  2.1773138739050565e-06
typ1 = "x"
Delta =  -2*np.pi*3.4*10**9

ini_sequence = sequence(np.array([photons(typ0,order0,delta0,t0),photons(typ1,order1,delta1,t1)]))

atom_number = 50

animate_system = cavity_atom_light_system(atom_number = atom_number, cooperativity = 1000, Delta = Delta, Omega = 0.2*10**6*2*np.pi)

# animate_system = cavity_atom_light_system(atom_number = 100, cooperativity = 56, Delta = -197*Gamma, Omega = 0.2*10**6*2*np.pi)
# animate_para1 = [0.0e-06,0.0e-06,1.593302185297957e-06,74,-423699254.15045,0.0e-06]
# animate_para2 = [0.0e-06,0.0e-06,0.0e-06,74,-423699254.15045,2.4134381601596233e-06]

time_step = 0.2*10**(-7)

num_frames = int((t0+t1)/time_step+1)

colors = []
states = []

for frame in range(num_frames):
    t = frame*time_step
    colors.append(animate_system.evolve_photons_t(ini_sequence,t))
    states.append(animate_system.psi)
    animate_system.system_reset()
    print("\r", end="")
    print(f"frame generated:{frame}/{num_frames}", end="")
    sys.stdout.flush()





frame generated:129/130

In [5]:
theta = np.linspace(0, np.pi, 100)
phi = np.linspace(0, np.pi*4/2, 100)
theta_grid, phi_grid = np.meshgrid(theta, phi)
y = np.sin(theta_grid) * np.cos(phi_grid)
x = -np.sin(theta_grid) * np.sin(phi_grid)
z = -np.cos(theta_grid)

image_list = []
for i in range(num_frames):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    q_func = colors[i]
    norm = Normalize(vmin=np.min(q_func), vmax=np.max(q_func))
    
    ax.plot_surface(x, y, z, rstride=1, cstride=1, facecolors=plt.cm.jet(norm(q_func)), alpha=0.3, linewidth=0, antialiased=False)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('Husimi-Q Representation')

    plt.savefig(f"fig/20231121_1stepGHZ{i}.jpg")
    plt.close()
    image_list.append(f"fig/20231121_1stepGHZ{i}.jpg")
    print("\r", end="")
    print(f"data generated:{i}/{num_frames}", end="")
    sys.stdout.flush()
    
gif_name = "fig/20231121_1stepGHZ.gif"
duration = 0.05
create_gif(image_list, gif_name,duration)

data generated:129/130

In [24]:
min_res_08_loss=np.array([0.118789,0.115254,0.112133,0.109125,0.106307,0.103659,0.101195,0.09883,0.096603,
0.094524,0.092478,0.09054,0.088724,0.086946,0.085308,0.083732,0.082226,0.080796,0.079388,0.078055,
0.076779,0.075569,0.074398,0.073281,0.072163,0.071091])

min_res_04_loss=np.array([0.120369,0.117276,0.113716,0.110908,0.108015,0.104744,0.102425,0.099800,0.097836,
0.095377,0.093450,0.091434,0.089640,0.087856,0.086324,0.084697,0.083106,0.081610,0.080242,0.078915,
0.077624,0.076384,0.075184,0.074029,0.072939,0.071878])

min_res_01_loss=np.array([0.128315,0.124657,0.121286,0.118051,0.115066,0.112250,0.109565,0.107058,0.104651,
0.102402,0.100255,0.098190,0.096268,0.094408,0.092621,0.090937,0.089281,0.087734,0.086251,0.084839,
0.083560,0.082110,0.080879,0.079626,0.078315,0.077282])


fit01 = np.polyfit(np.array([np.log(50+i*2) for i in range(26)]),np.log(min_res_01_loss),1)
fit04 = np.polyfit(np.array([np.log(50+i*2) for i in range(26)]),np.log(min_res_04_loss),1)
fit08 = np.polyfit(np.array([np.log(50+i*2) for i in range(26)]),np.log(min_res_08_loss),1)

print("1,loss:",fit01)
print("8, loss:",fit08)


fig,ax = plt.subplots()
ax.scatter(np.array([np.log(50+i*2) for i in range(26)]),np.log(min_res_01_loss),label = f"1 frequency with loss,scale as {round(fit01[0],2)}")
ax.scatter(np.array([np.log(50+i*2) for i in range(26)]),np.log(min_res_04_loss),label = f"4 frequencies with loss,scale as {round(fit08[0],2)}")
ax.scatter(np.array([np.log(50+i*2) for i in range(26)]),np.log(min_res_08_loss),label = f"8 frequencies with loss,scale as {round(fit08[0],2)}")

ax.set_xlabel("Ln(N)")
ax.set_ylabel("Ln(Xr)")
ax.set_title(f"Scale with loss")
ax.legend()
plt.show()

1,loss: [-0.73139727  0.80752283]
8, loss: [-0.740263    0.76438416]


In [12]:
SXsteps = np.array([1,2,3,4,5,6,7])
fidm1_10203040 = np.array([0.770289, 0.565149, 0.368263, 0.201661, 0.139975, 0.038862, 0.010960])
fidm1_1040 = np.array([0.760275, 0.344848, 0.258377, 0.049790, 0.014448, 0.001118, 0.000532])
fidm1_22 = 1-np.array([0.789205826631572, 0.8430817869939182, 0.9515106276540253, 0.9769325682556342, 0.9951521888567376, 0.9958183942013776, 0.9988511795611494])

fig,ax = plt.subplots()
ax.scatter(SXsteps,fidm1_1040,label = f"N10N40")
ax.scatter(SXsteps,fidm1_10203040,label = f"N10N20N30N40")
ax.scatter(SXsteps,fidm1_22,label = f"Dicke 22")

ax.set_xlabel("N")
ax.set_ylabel("1-Fidelity")
ax.set_title(f"Ideal 1-fidelities - steps number")
ax.legend()
plt.show()